In [67]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "8"  # Sustituye 8 por tus núcleos físicos


In [68]:
# CARGA DE DATOS 1 — setup en una sola celda
import sys, subprocess

def pip_install(*pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", *pkgs])

# 1) Arreglar conflictos típicos con conda: requests / urllib3 (versiones mínimas)
pip_install("requests>=2.31,<3", "urllib3>=2.2.2", "idna>=3.7", "charset-normalizer>=3.3")

# 2) Instalar librerías del proyecto
pip_install("understatapi", "pandas", "sqlalchemy")

print("✅ Dependencias listas. (Recomendado: reinicia el kernel tras esta celda)")

✅ Dependencias listas. (Recomendado: reinicia el kernel tras esta celda)


In [69]:
###ARCHIVO DATOS GENERALES POR JUGADOR


from understatapi import UnderstatClient
import pandas as pd

# Crear cliente
understat = UnderstatClient()


laliga_players_2025 = understat.league(league="La_Liga").get_player_data(season="2025")
df_players = pd.DataFrame(laliga_players_2025)
print(df_players.head())

#GUARDAMOS ARCHIVO 
df_players.to_csv("laliga_PLAYERS_2025.csv", index=False)

     id           player_name games time goals                  xG assists  \
0  3423  Kylian Mbappe-Lottin     5  449     5   5.010243654251099       1   
1  2543        Borja Iglesias     6  248     3  2.2630536183714867       0   
2  2589              Rafa Mir     4  242     3  0.5829365812242031       0   
3  4175            Pere Milla     4  216     3  2.0866236090660095       0   
4  8026              Raphinha     4  283     3   2.107767730951309       1   

                   xA shots key_passes yellow_cards red_cards position  \
0  1.3676733151078224    32         15            2         0        F   
1   0.249203784391284     7          5            0         0      F S   
2                   0     4          0            0         0      F S   
3  0.5759503245353699    12          2            0         1      M S   
4  1.1684775073081255    12          7            1         0      M S   

    team_title npg                npxG             xGChain           xGBuildup  
0  Re

In [70]:
###ARCHIVO TIROS POR JUGADOR


from understatapi import UnderstatClient
import pandas as pd, time, json

u = UnderstatClient()

# 1) Partidos de LaLiga 2025 (25/26)
matches = u.league("La_Liga").get_match_data(season="2025")

# 2) Sacar el id de partido como STRING
def get_mid_str(m):
    for k in ("id", "match_id", "matchId"):
        if m.get(k) is not None:
            return str(m[k])
    return None

match_ids = [get_mid_str(m) for m in matches if get_mid_str(m) is not None]

# 3) Descargar y normalizar shots por partido
all_shots = []
for mid in match_ids[:60]:                     # quita el slice cuando vaya bien
    try:
        raw = u.match(mid).get_shot_data()     # puede venir como str (JSON)
        data = json.loads(raw) if isinstance(raw, str) else raw

        # Understat suele devolver {'h': [...], 'a': [...]} para home/away
        if isinstance(data, dict) and ('h' in data or 'a' in data):
            for side_key, side_name in (('h','home'), ('a','away')):
                for s in data.get(side_key, []):
                    rec = dict(s)              # copia para no mutar el original
                    rec['match_id'] = mid
                    rec['side'] = side_name
                    all_shots.append(rec)
        elif isinstance(data, list):
            # por si tu versión devuelve lista directa
            for s in data:
                rec = dict(s)
                rec['match_id'] = mid
                all_shots.append(rec)
        else:
            print(f"match {mid}: formato inesperado {type(data)}")
        time.sleep(0.2)                        # cortesía
    except Exception as e:
        # Algunos ids pueden no estar disponibles (partidos futuros, etc.)
        print(f"fallo en match {mid}", e)

df_shots = pd.DataFrame(all_shots)
print(df_shots.shape)
df_shots.head()

#GUARDAMOS ARCHIVO DE TIROS
df_shots.to_csv("laliga_shots_2025.csv", index=False)
pd.DataFrame(matches).to_csv("laliga_matches_2025_raw.csv", index=False)

fallo en match 29204 29204 is not a valid match
fallo en match 29201 29201 is not a valid match
fallo en match 29199 29199 is not a valid match
fallo en match 29209 29209 is not a valid match
fallo en match 29210 29210 is not a valid match
fallo en match 29211 29211 is not a valid match
fallo en match 29212 29212 is not a valid match
fallo en match 29213 29213 is not a valid match
fallo en match 29214 29214 is not a valid match
fallo en match 29215 29215 is not a valid match
fallo en match 29216 29216 is not a valid match
fallo en match 29217 29217 is not a valid match
(1173, 21)


In [71]:
###ARCHIVO DATOS PARTIDOS

import pandas as pd
import numpy as np

# --- INFO base de partidos ---
matches_df = pd.DataFrame(matches).copy()
# normaliza match_id como string
if 'match_id' in matches_df.columns:
    matches_df['match_id'] = matches_df['match_id'].astype(str)
elif 'id' in matches_df.columns:
    matches_df['match_id'] = matches_df['id'].astype(str)

# --- Agregados tiro a tiro ---
s = df_shots.copy()

# asegurar tipos numéricos
for col in ['xG','X','Y']:
    if col in s.columns:
        s[col] = pd.to_numeric(s[col], errors='coerce')

s['on_target']  = s['result'].isin(['Goal','SavedShot']).astype(int)
s['goal']       = (s['result']=='Goal').astype(int)
s['open_play']  = (s['situation']=='OpenPlay').astype(int)
s['from_corner']= (s['situation']=='FromCorner').astype(int)
s['set_piece']  = s['situation'].isin(['FromCorner','SetPiece','DirectFreekick']).astype(int)
s['penalty']    = (s['situation']=='Penalty').astype(int)

s['xg_open']       = s['xG'] * s['open_play']
s['xg_setpieces']  = s['xG'] * s['set_piece']

agg = (s.groupby(['match_id','h_a'])
         .agg(
             shots        = ('id','count'),
             shots_ot     = ('on_target','sum'),
             goals_by_shot= ('goal','sum'),
             xg           = ('xG','sum'),
             xg_open      = ('xg_open','sum'),
             xg_setpieces = ('xg_setpieces','sum'),
             pens         = ('penalty','sum'),
             corner_shots = ('from_corner','sum'),
             avg_x        = ('X','mean'),
             avg_y        = ('Y','mean')
         )
         .reset_index())

wide = (agg.pivot(index='match_id', columns='h_a',
                  values=['shots','shots_ot','goals_by_shot','xg','xg_open','xg_setpieces','pens','corner_shots','avg_x','avg_y'])
          .rename(columns={'h':'home','a':'away'}))
wide.columns = [f"{a}_{b}" for a,b in wide.columns]
wide = wide.rename(columns={
    'shots_home':'home_shots', 'shots_away':'away_shots',
    'shots_ot_home':'home_shots_ot','shots_ot_away':'away_shots_ot',
    'goals_by_shot_home':'home_goals_by_shot','goals_by_shot_away':'away_goals_by_shot',
    'xg_home':'home_xg','xg_away':'away_xg',
    'xg_open_home':'home_xg_open','xg_open_away':'away_xg_open',
    'xg_setpieces_home':'home_xg_setpieces','xg_setpieces_away':'away_xg_setpieces',
    'pens_home':'home_pens','pens_away':'away_pens',
    'corner_shots_home':'home_corner_shots','corner_shots_away':'away_corner_shots',
    'avg_x_home':'home_avg_x','avg_x_away':'away_avg_x',
    'avg_y_home':'home_avg_y','avg_y_away':'away_avg_y'
})

# --- Info de equipos / fecha / marcador ---
# primero intentamos desde df_shots (suele venir con estas columnas)
base_cols = [c for c in ['match_id','h_team','a_team','date','h_goals','a_goals'] if c in s.columns]
if base_cols:
    info = s[base_cols].drop_duplicates('match_id').set_index('match_id')
else:
    # fallback a matches_df si df_shots no trae esas columnas en tu versión
    fallback_cols = [c for c in ['match_id','h_team','a_team','date','h_goals','a_goals','home_team','away_team','home_goals','away_goals'] if c in matches_df.columns]
    info = matches_df[fallback_cols].drop_duplicates('match_id').set_index('match_id')
    # normalizamos nombres más comunes
    info = info.rename(columns={'home_team':'h_team','away_team':'a_team','home_goals':'h_goals','away_goals':'a_goals'})

matches_features = (info.join(wide, how='left')
                        .reset_index()
                        .rename(columns={'h_goals':'home_goals','a_goals':'away_goals'}))


# --- Coerción de tipos para goles (evita dtype=object) ---
for c_from, c_to in [
    ('h_goals','home_goals'), ('a_goals','away_goals'),
    ('home_goals','home_goals'), ('away_goals','away_goals')
]:
    if c_from in matches_features.columns:
        matches_features[c_from] = pd.to_numeric(matches_features[c_from], errors='coerce')
if 'home_goals' in matches_features.columns:
    matches_features['home_goals'] = pd.to_numeric(matches_features['home_goals'], errors='coerce')
if 'away_goals' in matches_features.columns:
    matches_features['away_goals'] = pd.to_numeric(matches_features['away_goals'], errors='coerce')


# CSV
matches_features.to_csv("laliga_matches_features_2025.csv", index=False)

# Opcional: SQLite
from sqlalchemy import create_engine
engine = create_engine("sqlite:///football.db", echo=False)
matches_features.to_sql("laliga_2025_matches", engine, if_exists="replace", index=False)

print("✅ Guardado: laliga_matches_features_2025.csv y tabla SQLite.")

✅ Guardado: laliga_matches_features_2025.csv y tabla SQLite.


In [72]:
# ===== FALTAS POR JUGADOR (FBref → automático, robusto) =====
import sys, subprocess, pandas as pd, numpy as np, re, unicodedata, io

def _pip(*pkgs):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *pkgs])
    except Exception as e:
        print("⚠️ No pude instalar alguna dependencia:", e)

# deps de red/parseo
try:
    import requests
    from bs4 import BeautifulSoup, Comment
except Exception:
    _pip("requests>=2.31,<3", "beautifulsoup4>=4.12", "lxml>=4.9")
    import requests
    from bs4 import BeautifulSoup, Comment

def deaccent(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"\b(cf|fc|sd|sad|ud|cd)\b", "", s)
    s = s.replace(".", " ").replace(",", " ").replace("-", " ")
    s = s.replace("atletico de madrid","atletico madrid")
    s = s.replace("real betis balompie","real betis")
    s = s.replace("athletic club","athletic bilbao")
    s = s.replace("ud las palmas","las palmas")
    return re.sub(r"\s+"," ", s).strip()

def _flatten_columns(df):
    # Convierte MultiIndex de columnas en texto simple
    if isinstance(df.columns, pd.MultiIndex):
        new_cols = []
        for tup in df.columns:
            parts = [str(x) for x in tup if (x is not None and str(x).lower() != "nan")]
            name = " ".join(parts).strip()
            new_cols.append(name)
        df.columns = new_cols
    else:
        df.columns = [str(c) for c in df.columns]
    return df

def _pickcol(df, *cands):
    cands = [c.lower() for c in cands]
    for c in df.columns:
        lc = c.lower()
        if any(x in lc for x in cands):
            return c
    return None

def fetch_fbref_players_misc_laliga(season=2025):
    """Devuelve DF con player, team, fouls(Fls), fouls_drawn(Fld), yellows, reds, matches, nineties, fouls_pg."""
    yspan = f"{season}-{season+1}"
    urls = [
        f"https://fbref.com/en/comps/12/{yspan}/misc/{yspan}-La-Liga-Stats",
        f"https://fbref.com/en/comps/12/{yspan}/{yspan}-La-Liga-Stats",
    ]
    hdrs = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://fbref.com/"
    }

    for url in urls:
        try:
            r = requests.get(url, headers=hdrs, timeout=30)
            if r.status_code != 200:
                continue
            soup = BeautifulSoup(r.text, "lxml")

            # 1) Intenta extraer el bloque comentado que contiene la tabla con id="stats_misc"
            table_html = None
            for com in soup.find_all(string=lambda x: isinstance(x, Comment)):
                txt = str(com)
                if 'id="stats_misc"' in txt or "stats_misc" in txt:
                    table_html = txt
                    break

            tables = []
            if table_html:
                tables = pd.read_html(io.StringIO(table_html))
            if not tables:
                # 2) Fallback: leer todas las tablas de la página y elegir la que contenga "Fls" y "Player"
                all_tables = pd.read_html(r.text)
                for t in all_tables:
                    t = _flatten_columns(t.copy())
                    if (_pickcol(t, "player") and (_pickcol(t, "fls", "fouls") or _pickcol(t, "fld", "fouls drawn"))):
                        tables = [t]
                        break
            if not tables:
                continue

            df = tables[0].copy()
            df = _flatten_columns(df)

            # Hay filas "header" repetidas; eliminarlas
            pcol = _pickcol(df, "player")
            if pcol:
                df = df[df[pcol] != "Player"]

            # Renombrado flexible
            col_player = _pickcol(df, "player")
            col_team   = _pickcol(df, "squad", "team")
            col_fls    = _pickcol(df, "fls", "fouls committed")
            col_fld    = _pickcol(df, "fld", "fouls drawn")
            col_yel    = _pickcol(df, "crdy", "yellow")
            col_red    = _pickcol(df, "crdr", "red")
            col_mp     = _pickcol(df, "mp")        # Matches Played
            col_90s    = _pickcol(df, "90s")

            rename = {}
            if col_player: rename[col_player] = "player"
            if col_team:   rename[col_team]   = "team"
            if col_fls:    rename[col_fls]    = "fouls"
            if col_fld:    rename[col_fld]    = "fouls_drawn"
            if col_yel:    rename[col_yel]    = "yellows"
            if col_red:    rename[col_red]    = "reds"
            if col_mp:     rename[col_mp]     = "matches"
            if col_90s:    rename[col_90s]    = "nineties"
            df = df.rename(columns=rename)

            # Asegura tipos
            for c in ["fouls","fouls_drawn","yellows","reds","matches","nineties"]:
                if c in df.columns:
                    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

            # Medias por partido (si hay matches)
            if "matches" in df.columns and (df["matches"]>0).any():
                df["fouls_pg"]   = (df["fouls"]/df["matches"]).replace([np.inf,-np.inf], np.nan).round(3)
                df["yellows_pg"] = (df["yellows"]/df["matches"]).replace([np.inf,-np.inf], np.nan).round(3)
                df["reds_pg"]    = (df["reds"]/df["matches"]).replace([np.inf,-np.inf], np.nan).round(3)

            if "team" in df.columns:
                df["team_norm"] = df["team"].map(deaccent)
            else:
                df["team_norm"] = np.nan
            df["player_norm"] = df.get("player", pd.Series(dtype=str)).astype(str).str.lower().str.strip()

            keep = [c for c in ["player","team","team_norm","fouls","fouls_pg","fouls_drawn","yellows","reds","matches"] if c in df.columns]
            return df[keep]
        except Exception as e:
            # Intenta siguiente URL
            continue

    return pd.DataFrame()

df_player_fouls = fetch_fbref_players_misc_laliga(2025)
print("df_player_fouls shape:", df_player_fouls.shape)
display(df_player_fouls.head(10))



df_player_fouls shape: (448, 7)


,player,team,team_norm,fouls,fouls_drawn,yellows,reds
0,Jones El-Abdellaoui,Celta Vigo,celta vigo,0,0,0,0
1,Abdel Abqar,Getafe,getafe,1,2,0,0
2,Akor Adams,Sevilla,sevilla,2,0,0,0
3,David Affengruber,Elche,elche,5,8,0,0
4,Julen Agirrezabala,Valencia,valencia,0,0,0,0
5,Lucien Agoume,Sevilla,sevilla,10,1,2,0
6,Marc Aguado,Elche,elche,2,3,0,0
7,Lucas Ahijado,Oviedo,oviedo,2,1,1,0
8,Ilias Akhomach,Villarreal,villarreal,2,1,0,0
9,David Alaba,Real Madrid,real madrid,0,0,0,0


In [73]:
# ==== UTILIDADES GENERALES (colocar al inicio del notebook) ====
import os, io, glob, json, math, warnings
import numpy as np
import pandas as pd
from unicodedata import normalize

def deaccent(s):
    """Quita tildes/acentos y normaliza espacios. Devuelve str o NaN si no aplica."""
    if s is None or (isinstance(s, float) and math.isnan(s)): 
        return s
    s = str(s)
    s = normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
    return " ".join(s.split())

def _read_any(path):
    """Lee CSV/Parquet según extensión. Retorna DataFrame vacío si falla."""
    try:
        ext = os.path.splitext(path)[1].lower()
        if ext in [".csv", ".tsv"]:
            sep = "\t" if ext == ".tsv" else ","
            return pd.read_csv(path, sep=sep, low_memory=False, encoding="utf-8")
        if ext in [".parquet", ".pq"]:
            return pd.read_parquet(path)
        # fallback: intentar como CSV
        return pd.read_csv(path, low_memory=False, encoding="utf-8")
    except Exception as e:
        warnings.warn(f"No se pudo leer {path}: {e}")
        return pd.DataFrame()

def load_first_existing(candidates, extra_dirs=("data", "datasets", ".")):
    """
    Dada una lista de nombres/paths, devuelve el primer DataFrame legible.
    Busca en los directorios de extra_dirs si el nombre no es un path absoluto.
    Si no encuentra nada, devuelve DataFrame vacío.
    """
    # Acepta string o lista
    if isinstance(candidates, (str, os.PathLike)):
        candidates = [str(candidates)]
    tried = []
    for name in candidates:
        # Si es path absoluto o relativo existente, probar directo
        if os.path.exists(name):
            df = _read_any(name)
            if not df.empty or df.shape[0] > 0 or df.shape[1] > 0:
                print(f"Cargado: {name}  -> {df.shape}")
                return df
            tried.append(name)
        # Probar en carpetas auxiliares
        for d in extra_dirs:
            p = os.path.join(d, name)
            if os.path.exists(p):
                df = _read_any(p)
                if not df.empty or df.shape[0] > 0 or df.shape[1] > 0:
                    print(f"Cargado: {p}  -> {df.shape}")
                    return df
                tried.append(p)
        # Probar con comodines (ej. mayúsculas/minúsculas o sufijos)
        base, ext = os.path.splitext(name)
        if not ext:
            patterns = [f"{base}.*"]
        else:
            patterns = [name]
        for d in extra_dirs:
            for pat in patterns:
                for p in glob.glob(os.path.join(d, pat)):
                    if os.path.isfile(p):
                        df = _read_any(p)
                        if not df.empty or df.shape[0] > 0 or df.shape[1] > 0:
                            print(f"Cargado (match): {p}  -> {df.shape}")
                            return df
                        tried.append(p)
    warnings.warn(f"No se encontró ningún archivo de {candidates}. Intentados: {tried[:5]}{' ...' if len(tried)>5 else ''}")
    return pd.DataFrame()


In [74]:
# --- AÑADIDO 1: cargar la tabla de temporada por jugador (Understat) ----
df_players_season_raw = load_first_existing([
    "laliga_PLAYERS_2025.csv", "laliga_players_2025.csv", "LALIGA_PLAYERS_2025.csv"
])

def pick(df, *cands):
    for c in df.columns:
        lc = str(c).lower()
        for k in cands:
            if lc == k or k in lc:
                return c
    return None

def std_players_season(df):
    if df.empty: return df
    out = df.copy()
    # columnas posibles en understat
    col_player = pick(out, "player", "player_name", "name")
    col_team   = pick(out, "team", "team_title", "squad")
    col_ast    = pick(out, "assists", "ast")
    col_xa     = pick(out, "xa", "x_a", "expected assists")
    col_yel    = pick(out, "yellow", "crdy")
    col_red    = pick(out, "red", "crdr")
    rename = {col_player:"player", col_team:"team", col_ast:"assists", col_xa:"xA",
              col_yel:"yellows", col_red:"reds"}
    rename = {k:v for k,v in rename.items() if k}
    out = out.rename(columns=rename)
    out["team_norm"] = out["team"].map(deaccent) if "team" in out.columns else None
    keep = [c for c in ["player","team","team_norm","assists","xA","yellows","reds"] if c in out.columns]
    return out[keep]

df_players_season = std_players_season(df_players_season_raw)

# --- AÑADIDO 2 (REEMPLAZO COMPLETO): agregados de tiros por jugador ----------
# Asegura que df_player_shots exista (lo cargamos si no está definido todavía)
try:
    df_player_shots
except NameError:
    df_player_shots = load_first_existing(["player_match.csv","laliga_player_match_2025.csv"])

agg_shots_players = pd.DataFrame()

# Cargamos tiros crudos una vez (fallback y para detectar home/away)
shots_raw = load_first_existing(["laliga_shots_2025.csv"])

if (isinstance(df_player_shots, pd.DataFrame)
    and not df_player_shots.empty
    and {'team','player'}.issubset(df_player_shots.columns)):

    ps = df_player_shots.copy()

    # Normaliza xG si viene como 'xg'
    if 'xg' in ps.columns and 'xG' not in ps.columns:
        ps = ps.rename(columns={'xg':'xG'})

    # Si tenemos h_team/a_team podemos separar goles casa/fuera
    if {'h_team','a_team','goals'}.issubset(ps.columns):
        ps['side'] = np.where(ps['team']==ps['h_team'], 'home',
                       np.where(ps['team']==ps['a_team'], 'away', None))
        ps['goals_home'] = np.where(ps['side']=='home', ps.get('goals',0), 0)
        ps['goals_away'] = np.where(ps['side']=='away', ps.get('goals',0), 0)

    # Sumamos por jugador-equipo todo lo que exista
    cols_to_sum = [c for c in ['shots','shots_ot','goals','xG','goals_home','goals_away'] if c in ps.columns]
    if cols_to_sum:
        agg_shots_players = (ps.groupby(['team','player'], as_index=False)
                               .agg(**{k:(k,'sum') for k in cols_to_sum}))

# Fallback: construir desde tiros crudos si no hay player_match agregado
if agg_shots_players.empty and (not shots_raw.empty) and \
   {'result','h_a','h_team','a_team','player','id'}.issubset(shots_raw.columns):

    s = shots_raw.copy()
    s['on_target']  = s['result'].isin(['Goal','SavedShot']).astype(int)
    s['is_goal']    = (s['result']=='Goal').astype(int)
    s['team']       = np.where(s['h_a']=='h', s['h_team'],
                        np.where(s['h_a']=='a', s['a_team'], None))
    s['goals_home'] = np.where(s['h_a']=='h', s['is_goal'], 0)
    s['goals_away'] = np.where(s['h_a']=='a', s['is_goal'], 0)

    if 'xG' not in s.columns and 'xg' in s.columns:
        s = s.rename(columns={'xg':'xG'})
    if 'xG' not in s.columns:
        s['xG'] = 0.0  # por seguridad

    agg_shots_players = (s.groupby(['team','player'], as_index=False)
                           .agg(shots=('id','count'),
                                shots_ot=('on_target','sum'),
                                goals=('is_goal','sum'),
                                xG=('xG','sum'),
                                goals_home=('goals_home','sum'),
                                goals_away=('goals_away','sum')))

# Si aún falta xG, crea la columna a 0.0 para evitar errores aguas abajo
if not agg_shots_players.empty and 'xG' not in agg_shots_players.columns:
    agg_shots_players['xG'] = 0.0



Cargado: laliga_PLAYERS_2025.csv  -> (446, 18)
Cargado: laliga_shots_2025.csv  -> (1173, 21)


In [75]:
if not agg_shots_players.empty:
    agg_shots_players.to_csv("agg_shots_players_2025.csv", index=False)


In [76]:
# ===================== DASHBOARD DE EQUIPOS (todo en una celda) =====================
import os, unicodedata, re
import pandas as pd
import numpy as np

# --------------------------- utilidades base ---------------------------------
def load_first_existing(paths):
    for p in paths:
        if p and os.path.exists(p):
            return pd.read_csv(p)
    return pd.DataFrame()

def deaccent(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"\b(cf|fc|sd|sad|ud|cd)\b", "", s)
    s = s.replace(".", " ").replace(",", " ").replace("-", " ")
    s = s.replace("atletico de madrid","atletico madrid")
    s = s.replace("real betis balompie","real betis")
    s = s.replace("athletic club","athletic bilbao")
    s = s.replace("ud las palmas","las palmas")
    return re.sub(r"\s+"," ", s).strip()

def pick_col(df, *patterns):
    pats = [p.lower() for p in patterns]
    for c in df.columns:
        lc = c.lower()
        if any(p in lc for p in pats):
            return c
    return None

# --------------------------- carga datasets ----------------------------------
df_team0 = load_first_existing([
    "laliga_matches_with_fbref_2025.csv",
    "laliga_matches_features_2025.csv"
])
if df_team0.empty:
    raise SystemExit("No encuentro 'laliga_matches_with_fbref_2025.csv' ni 'laliga_matches_features_2025.csv'.")

df_team = df_team0.copy()

# Jugador-temporada (Understat league player data: asistencias, xA, amarillas, rojas, key passes)
df_players_season_raw = load_first_existing([
    "laliga_PLAYERS_2025.csv", "laliga_players_2025.csv", "LALIGA_PLAYERS_2025.csv"
])

def std_players_season(df):
    if df.empty: return df
    out = df.copy()
    def pick(df, *cands):
        for c in df.columns:
            lc = str(c).lower()
            for k in cands:
                if lc == k or k in lc:
                    return c
        return None
    rename = {}
    for src, dst in [
        (pick(out,"player","player_name","name"),       "player"),
        (pick(out,"team","team_title","squad"),         "team"),
        (pick(out,"assists","ast"),                     "assists"),
        (pick(out,"xa","x_a","expected assists"),       "xA"),
        (pick(out,"yellow","crdy"),                     "yellows"),
        (pick(out,"red","crdr"),                        "reds"),
        (pick(out,"key_passes","key passes","keypass","kp","keypasses"), "key_passes"),  # NUEVO
    ]:
        if src: rename[src] = dst
    out = out.rename(columns=rename)
    if "team" in out.columns:
        out["team_norm"] = out["team"].map(deaccent)
    keep = [c for c in ["player","team","team_norm","assists","xA","yellows","reds","key_passes"] if c in out.columns]
    return out[keep]

df_players_season = std_players_season(df_players_season_raw)

# Jugador-partido (tiros) agregado por jugador
AGG_FILE = "agg_shots_players_2025.csv"
agg_shots_players = load_first_existing([AGG_FILE])

if agg_shots_players.empty:
    df_player_shots = load_first_existing(["player_match.csv","laliga_player_match_2025.csv"])
    if not df_player_shots.empty and {'team','player','shots'}.issubset(df_player_shots.columns):
        ps = df_player_shots.copy()
        if 'xg' in ps.columns and 'xG' not in ps.columns:
            ps.rename(columns={'xg':'xG'}, inplace=True)
        if {'h_team','a_team','goals'}.issubset(ps.columns):
            ps["side"] = np.where(ps["team"]==ps["h_team"], "home",
                           np.where(ps["team"]==ps["a_team"], "away", None))
            ps["goals_home"] = np.where(ps["side"]=="home", ps["goals"], 0)
            ps["goals_away"] = np.where(ps["side"]=="away", ps["goals"], 0)
        agg_shots_players = (ps.groupby(["team","player"], as_index=False)
                               .agg(**{k:(k,'sum') for k in [c for c in ["shots","shots_ot","goals","xG","goals_home","goals_away"] if c in ps.columns]}))
    else:
        shots_raw = load_first_existing(["laliga_shots_2025.csv"])
        if not shots_raw.empty and {'result','h_a','h_team','a_team','player','id'}.issubset(shots_raw.columns):
            s = shots_raw.copy()
            s['on_target']  = s['result'].isin(['Goal','SavedShot']).astype(int)
            s['is_goal']    = (s['result']=='Goal').astype(int)
            s['team']       = np.where(s['h_a']=='h', s['h_team'],
                                np.where(s['h_a']=='a', s['a_team'], None))
            s['goals_home'] = np.where(s['h_a']=='h', s['is_goal'], 0)
            s['goals_away'] = np.where(s['h_a']=='a', s['is_goal'], 0)
            agg_shots_players = (s.groupby(['team','player'], as_index=False)
                                   .agg(shots=('id','count'),
                                        shots_ot=('on_target','sum'),
                                        goals=('is_goal','sum'),
                                        xG=('xG','sum') if 'xG' in s.columns else ('is_goal','sum'),
                                        goals_home=('goals_home','sum'),
                                        goals_away=('goals_away','sum')))
if not agg_shots_players.empty and 'xg' in agg_shots_players.columns and 'xG' not in agg_shots_players.columns:
    agg_shots_players.rename(columns={'xg':'xG'}, inplace=True)

# ----------------- preparar df_team → formato largo con más campos ----------
must = ['h_team','a_team','home_goals','away_goals']
missing = [c for c in must if c not in df_team.columns]
if missing:
    raise SystemExit(f"Faltan columnas clave en el CSV de equipos: {missing}")

if 'date' in df_team.columns:
    df_team['date'] = pd.to_datetime(df_team['date'], errors='coerce')

home_sot  = pick_col(df_team, "home_shots_on_target","home_sot","home_shots_ot")
away_sot  = pick_col(df_team, "away_shots_on_target","away_sot","away_shots_ot")
home_pens = pick_col(df_team, "home_pen","home_pens","home_penalty","home_penalties")
away_pens = pick_col(df_team, "away_pen","away_pens","away_penalty","away_penalties")
home_corn = pick_col(df_team, "home_corners","home_corner")
away_corn = pick_col(df_team, "away_corners","away_corner")

home_cols = {
    'team':'h_team', 'opponent':'a_team',
    'goals':'home_goals', 'opp_goals':'away_goals',
    'shots':'home_shots',
}
away_cols = {
    'team':'a_team', 'opponent':'h_team',
    'goals':'away_goals', 'opp_goals':'home_goals',
    'shots':'away_shots',
}
if home_sot:  home_cols['shots_ot'] = home_sot
if away_sot:  away_cols['shots_ot'] = away_sot
if home_pens: home_cols['pens']     = home_pens
if away_pens: away_cols['pens']     = away_pens
if home_corn: home_cols['corners']  = home_corn
if away_corn: away_cols['corners']  = away_corn

def make_side(df, mapper, side):
    cols = {k:v for k,v in mapper.items() if v in df.columns}
    base = (['date'] if 'date' in df.columns else []) + list(cols.values())
    out = df[base].copy()
    out = out.rename(columns={v:k for k,v in cols.items()})
    out['side'] = side
    return out

df_home = make_side(df_team, home_cols, 'home')
df_away = make_side(df_team, away_cols, 'away')
teams_long = pd.concat([df_home, df_away], ignore_index=True, sort=False)
teams_long['team_norm'] = teams_long['team'].map(deaccent)
teams_long['opp_norm']  = teams_long['opponent'].map(deaccent)

# ----------------- CLASIFICACIÓN (posición) ---------------------------------
def compute_standings(df_long):
    t = df_long.copy()
    t['goals'] = pd.to_numeric(t.get('goals'), errors='coerce')
    t['opp_goals'] = pd.to_numeric(t.get('opp_goals'), errors='coerce')
    t = t.dropna(subset=['goals','opp_goals'])
    pts = np.where(t['goals']>t['opp_goals'], 3, np.where(t['goals']==t['opp_goals'], 1, 0))
    table = (t.assign(pts=pts)
               .groupby('team', as_index=False)
               .agg(points=('pts','sum'), gf=('goals','sum'), ga=('opp_goals','sum')))
    table['gd'] = table['gf'] - table['ga']
    table = table.sort_values(['points','gd','gf'], ascending=False, kind='mergesort').reset_index(drop=True)
    table['rank'] = np.arange(1, len(table)+1)
    return table

standings_df = compute_standings(teams_long)

# ----------- PARTIDOS por jugador (para medias “por partido”) ----------------
def compute_games_per_player():
    pm = load_first_existing(["player_match.csv","laliga_player_match_2025.csv"])
    if not pm.empty and 'match_id' in pm.columns:
        base = pm.copy()
        if 'team' not in base.columns and {'h_team','a_team','side'}.issubset(base.columns):
            base['team'] = np.where(base['side']=='home', base['h_team'],
                             np.where(base['side']=='away', base['a_team'], np.nan))
        if {'team','player','match_id'}.issubset(base.columns):
            if {'h_team','a_team'}.issubset(base.columns):
                base['side'] = np.where(base['team']==base['h_team'],'home',
                                 np.where(base['team']==base['a_team'],'away',None))
                gp = (base[['team','player','match_id','side']].dropna()
                        .drop_duplicates()
                        .groupby(['team','player'], as_index=False)
                        .agg(games=('match_id','nunique'),
                             games_home=('side', lambda x: (x=='home').sum()),
                             games_away=('side', lambda x: (x=='away').sum())))
            else:
                gp = (base[['team','player','match_id']].dropna()
                        .drop_duplicates()
                        .groupby(['team','player'], as_index=False)
                        .agg(games=('match_id','nunique')))
            return gp

    # fallback: derivar desde tiros crudos
    shots_raw = load_first_existing(["laliga_shots_2025.csv"])
    if not shots_raw.empty and {'match_id','player','h_team','a_team','h_a'}.issubset(shots_raw.columns):
        s = shots_raw[['match_id','player','h_team','a_team','h_a']].drop_duplicates()
        s['team'] = np.where(s['h_a']=='h', s['h_team'], np.where(s['h_a']=='a', s['a_team'], None))
        s['side'] = np.where(s['h_a']=='h','home',np.where(s['h_a']=='a','away',None))
        gp = (s.groupby(['team','player'], as_index=False)
                .agg(games=('match_id','nunique'),
                     games_home=('side', lambda x: (x=='home').sum()),
                     games_away=('side', lambda x: (x=='away').sum())))
        return gp

    return pd.DataFrame(columns=['team','player','games','games_home','games_away'])

games_per_player = compute_games_per_player()

# ---------------------- agregadores para el resumen -------------------------
def team_summary(df_long, team):
    tdf = df_long[df_long['team'] == team].copy()
    res = {}

    # medias globales (se incluyen si existen)
    for col in ['goals','shots','shots_ot','corners','pens','fouls','yellows','reds']:
        if col in tdf.columns and tdf[col].notna().any():
            res[f"media_{col}_por_partido"] = round(pd.to_numeric(tdf[col], errors="coerce").mean(), 3)

    # medias home/away
    for col in ['goals','shots','shots_ot','corners','pens','fouls','yellows','reds']:
        if col in tdf.columns:
            if (tdf['side']=='home').any():
                res[f"media_home_{col}"] = round(pd.to_numeric(tdf.loc[tdf['side']=='home', col], errors="coerce").mean(), 3)
            if (tdf['side']=='away').any():
                res[f"media_away_{col}"] = round(pd.to_numeric(tdf.loc[tdf['side']=='away', col], errors="coerce").mean(), 3)

    # posición en la clasificación
    if 'standings_df' in globals():
        pos = standings_df.loc[standings_df['team']==team, 'rank']
        if not pos.empty:
            res['posicion'] = int(pos.iloc[0])

    res['partidos_conteo'] = int(tdf.shape[0])
    return pd.DataFrame(res, index=[team])

def last_results(df_long, team, k=5):
    t = df_long[df_long['team']==team].copy()
    if 'date' in t.columns:
        t = t.sort_values('date')
    def row_res(r):
        g, og = r.get('goals'), r.get('opp_goals')
        if pd.isna(g) or pd.isna(og): return "?"
        rch = 'W' if g>og else ('L' if g<og else 'D')
        side = 'H' if r.get('side')=='home' else 'A'
        d = r.get('date')
        d = pd.to_datetime(d).strftime('%Y-%m-%d') if pd.notna(d) else ''
        return f"{d} {rch} {int(g)}-{int(og)} vs {r.get('opponent')} ({side})"
    last = [row_res(r) for _, r in t.tail(k).iterrows()]
    def current_unbeaten_streak(x):
        cnt = 0
        for _, r in x.sort_values('date').iloc[::-1].iterrows():
            g, og = pd.to_numeric(r.get('goals'), errors='coerce'), pd.to_numeric(r.get('opp_goals'), errors='coerce')
            if pd.isna(g) or pd.isna(og): break
            if g >= og: cnt += 1
            else: break
        return cnt
    home_unbeaten = current_unbeaten_streak(t[t['side']=='home']) if not t.empty else 0
    away_unbeaten = current_unbeaten_streak(t[t['side']=='away']) if not t.empty else 0
    return last, home_unbeaten, away_unbeaten

# ------------------ tablas combinadas por equipo (para el panel) ------------
def top_tables(team, n=5):
    out = {}
    # ATAQUE (tiros Understat)
    if not agg_shots_players.empty:
        f = agg_shots_players[agg_shots_players["team"] == team].copy()
        if not f.empty:
            for c in ["shots","shots_ot","goals","xG","goals_home","goals_away"]:
                if c in f.columns:
                    f[c] = pd.to_numeric(f[c], errors="coerce").fillna(0)
            # merge con partidos por jugador (para promedios)
            if not games_per_player.empty:
                f = f.merge(games_per_player, on=['team','player'], how='left')

            # tiros + on target (+ por partido si hay games)
            cols = ["player","shots"]
            if "shots_ot" in f.columns: cols.append("shots_ot")
            if "games" in f.columns:    cols.append("games")
            tbl = f.sort_values("shots", ascending=False).head(n)[cols].copy()
            if "games" in tbl.columns:
                tbl["shots_pg"] = (tbl["shots"]/tbl["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                if "shots_ot" in tbl.columns:
                    tbl["shots_on_target_pg"] = (tbl["shots_ot"]/tbl["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                tbl = tbl.drop(columns=["games"])
            tbl = tbl.rename(columns={"shots_ot":"shots_on_target"})
            out["tiros"] = tbl

            # goles + xG (+ por partido)
            cols = [c for c in ["player","goals","xG","games"] if c in f.columns]
            if set(cols) >= {"player","goals"}:
                gxa = f.sort_values("goals", ascending=False).head(n)[cols].copy()
                if "xG" in gxa.columns: gxa["xG"] = pd.to_numeric(gxa["xG"], errors="coerce").round(3)
                if "games" in gxa.columns:
                    gxa["goals_pg"] = (gxa["goals"]/gxa["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    if "xG" in gxa.columns:
                        gxa["xG_pg"] = (gxa["xG"]/gxa["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    gxa = gxa.drop(columns=["games"])
                out["goles_xg"] = gxa

            # casa / fuera (+ por partido home/away si tenemos games_home/games_away)
            if {"goals_home","goals_away"}.issubset(f.columns):
                f["goals_total"] = f["goals_home"] + f["goals_away"]
                cols = ["player","goals_home","goals_away"]
                if {"games_home","games_away"}.issubset(f.columns):
                    cols += ["games_home","games_away"]
                gh = f.sort_values("goals_total", ascending=False).head(n)[cols].copy()
                if {"games_home","games_away"}.issubset(gh.columns):
                    gh["goals_home_pg"] = (gh["goals_home"]/gh["games_home"]).replace([np.inf,-np.inf], np.nan).round(2)
                    gh["goals_away_pg"] = (gh["goals_away"]/gh["games_away"]).replace([np.inf,-np.inf], np.nan).round(2)
                    gh = gh.drop(columns=["games_home","games_away"])
                out["casa_fuera"] = gh

    # Asistencias + xA; Amarillas + Rojas (tabla de temporada) + por partido
    if not df_players_season.empty and "team_norm" in df_players_season.columns:
        tnorm = deaccent(team)
        ps = df_players_season[df_players_season["team_norm"] == tnorm].copy()
        if not ps.empty:
            for c in ["assists","xA","yellows","reds","key_passes"]:
                if c in ps.columns: ps[c] = pd.to_numeric(ps[c], errors="coerce").fillna(0)
            if not games_per_player.empty:
                ps = ps.merge(games_per_player[['team','player','games']], on=['team','player'], how='left')

            # Asistencias/xA
            cols = [c for c in ["player","assists","xA","games"] if c in ps.columns]
            if set(cols) >= {"player","assists"}:
                ax = ps.sort_values("assists", ascending=False).head(n)[cols].copy()
                if "games" in ax.columns:
                    ax["assists_pg"] = (ax["assists"]/ax["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    if "xA" in ax.columns:
                        ax["xA_pg"] = (ax["xA"]/ax["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    ax = ax.drop(columns=["games"])
                out["asist_xa"] = ax

            # Key passes / por partido (NUEVO)
            if "key_passes" in ps.columns:
                cols = [c for c in ["player","key_passes","games"] if c in ps.columns]
                kp = ps.sort_values("key_passes", ascending=False).head(n)[cols].copy()
                if "games" in kp.columns:
                    kp["key_passes_pg"] = (kp["key_passes"]/kp["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    kp = kp.drop(columns=["games"])
                out["key_passes"] = kp.rename(columns={"key_passes":"key_passes_totales"})

            # Tarjetas
            if {"yellows","reds"}.issubset(ps.columns):
                cols = ["player","yellows","reds"] + (["games"] if "games" in ps.columns else [])
                cards = ps.sort_values(["yellows","reds"], ascending=False).head(n)[cols].copy()
                if "games" in cards.columns:
                    cards["yellows_pg"] = (cards["yellows"]/cards["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    cards["reds_pg"] = (cards["reds"]/cards["games"]).replace([np.inf,-np.inf], np.nan).round(2)
                    cards = cards.drop(columns=["games"])
                out["cards"] = cards

        # FALTAS por jugador (desde df_player_fouls)  ─ siempre con 'faltas_por_partido'
    if 'df_player_fouls' in globals() and isinstance(df_player_fouls, pd.DataFrame) and not df_player_fouls.empty:
        tnorm = deaccent(team)
        pf = df_player_fouls[df_player_fouls.get('team_norm','').astype(str) == tnorm].copy()
        if not pf.empty:
            # Tipos numéricos
            for c in ["fouls", "fouls_drawn", "matches"]:
                if c in pf.columns:
                    pf[c] = pd.to_numeric(pf[c], errors="coerce").fillna(0)

            # 1) Si el scrape trae 'matches', úsalo
            if "matches" in pf.columns and (pf["matches"] > 0).any():
                pf["fouls_pg"] = (pf["fouls"] / pf["matches"]).replace([np.inf, -np.inf], np.nan)

            # 2) Si NO hay 'matches', calcula con games_per_player
            if "fouls_pg" not in pf.columns and not games_per_player.empty:
                gpp = games_per_player.copy()
                gpp["team_norm"] = gpp["team"].map(deaccent)
                gpp = gpp[gpp["team_norm"] == tnorm][["player", "games"]]
                pf = pf.merge(gpp, on="player", how="left")
                if "games" in pf.columns and pf["games"].notna().any():
                    pf["fouls_pg"] = (pf["fouls"] / pf["games"]).replace([np.inf, -np.inf], np.nan)
                pf.drop(columns=[c for c in ["games"] if c in pf.columns], inplace=True)

            # Redondeo y columnas a mostrar
            if "fouls_pg" in pf.columns:
                pf["fouls_pg"] = pf["fouls_pg"].round(2)

            cols = ["player", "fouls"]
            if "fouls_pg" in pf.columns: cols.append("fouls_pg")
            if "fouls_drawn" in pf.columns: cols.append("fouls_drawn")

            out["faltas"] = (
                pf.sort_values(["fouls", "fouls_drawn"] if "fouls_drawn" in pf.columns else ["fouls"],
                               ascending=False)
                  .head(n)[cols]
                  .rename(columns={
                      "fouls": "faltas",
                      "fouls_pg": "faltas_por_partido",
                      "fouls_drawn": "faltas_recibidas",
                  })
            )


    return out

# ----------------------------- UI (ipywidgets) ------------------------------
try:
    import ipywidgets as W
    from IPython.display import display, HTML, clear_output
    use_widgets = True
except Exception:
    from IPython.display import display, HTML, clear_output
    use_widgets = False

equipos = sorted(teams_long['team'].dropna().unique().tolist())

def render_team_block(team, color="#e9eff7"):
    from IPython.display import HTML
    summ = team_summary(teams_long, team)
    tabs = top_tables(team, n=5)
    last5, home_unb, away_unb = last_results(teams_long, team, k=5)

    def df_to_html(df):
        return df.to_html(index=False) if isinstance(df, pd.DataFrame) and not df.empty else "<i>No disponible</i>"

    s = summ.iloc[0].to_dict()

    # métricas en columnas de 5 (incluye posición y todas las disponibles)
    def metrics_grid(s, per_col=5):
        entries = []
        def add(label, key):
            if key in s and pd.notna(s[key]): entries.append((label, s[key]))

        add("Posición", "posicion")
        add("Media Shots Por Partido", "media_shots_por_partido")
        add("Media Goals Por Partido", "media_goals_por_partido")
        add("Media Shots Ot Por Partido", "media_shots_ot_por_partido")
        add("Media Pens Por Partido", "media_pens_por_partido")
        add("Media Corners Por Partido", "media_corners_por_partido")
        add("Media Fouls Por Partido", "media_fouls_por_partido")
        add("Media Yellows Por Partido", "media_yellows_por_partido")
        add("Media Reds Por Partido", "media_reds_por_partido")
        add("Partidos Conteo", "partidos_conteo")

        for metric, label in [
            ('goals','Goals'), ('shots','Shots'), ('shots_ot','Shots OT'),
            ('pens','Pens'), ('corners','Corners'), ('fouls','Fouls'),
            ('yellows','Yellows'), ('reds','Reds')
        ]:
            add(f"Media Home {label}", f"media_home_{metric}")
            add(f"Media Away {label}", f"media_away_{metric}")

        if not entries: return ""
        cols = [entries[i:i+per_col] for i in range(0, len(entries), per_col)]
        out = ["<div style='display:flex;gap:24px;flex-wrap:wrap'>"]
        for col in cols:
            out.append("<ul style='margin:0 0 12px 18px;min-width:260px'>")
            for label, val in col: out.append(f"<li><b>{label}:</b> {val}</li>")
            out.append("</ul>")
        out.append("</div>")
        return "".join(out)

    html = [f"<div style='background:{color};padding:12px;border-radius:12px;margin-bottom:8px'>",
            f"<h3 style='margin:0 0 8px 0'>{team}</h3>",
            metrics_grid(s, per_col=5)]

    # Últimos 5 + rachas
    html.append("<div style='margin:6px 0 12px 0'><b>Últimos 5 resultados</b><ul style='margin:6px 0 0 18px'>")
    for line in last5: html.append(f"<li>{line}</li>")
    html.append("</ul>")
    html.append(f"<div style='margin-top:6px'><b>Racha sin perder (casa):</b> {home_unb} &nbsp;|&nbsp; "
                f"<b>Racha sin perder (fuera):</b> {away_unb}</div></div>")

    # Tablas combinadas (con columnas por partido si hay datos de games)
    html.append("<div style='display:flex;gap:16px;flex-wrap:wrap'>")
    html.append("<div><b>Tiros / On Target</b><br>"+df_to_html(tabs.get("tiros"))+"</div>")
    html.append("<div><b>Goles / xG</b><br>"+df_to_html(tabs.get("goles_xg"))+"</div>")
    html.append("<div><b>Goles (casa / fuera)</b><br>"+df_to_html(tabs.get("casa_fuera"))+"</div>")
    html.append("<div><b>Asistencias / xA</b><br>"+df_to_html(tabs.get("asist_xa"))+"</div>")
    html.append("<div><b>Key passes / por partido</b><br>"+df_to_html(tabs.get("key_passes"))+"</div>")
    html.append("<div><b>Amarillas / Rojas</b><br>"+df_to_html(tabs.get("cards"))+"</div>")
    html.append("<div><b>Faltas (cometidas / por partido)</b><br>"+df_to_html(tabs.get("faltas"))+"</div>")
    html.append("</div></div>")
    return HTML("".join(html))

def render_compare(teamA, teamB):
    box = HTML("<h2 style='margin:6px 0 12px 0'>Resumen comparado</h2>")
    left  = render_team_block(teamA, "#f6f9ff")
    right = render_team_block(teamB, "#fff6f6")
    display(box, HTML("<div style='display:flex;gap:16px;flex-wrap:wrap'>"), left, right, HTML("</div>"))
    miss = [b for b in ['corners','pens','shots_ot'] if b not in teams_long.columns]
    if miss:
        display(HTML(
            f"<p style='color:#a00'><b>Nota:</b> faltan columnas {miss} en tu CSV de equipos. "
            f"Si las añades (FBref/FotMob/tu pipeline) aparecerán automáticamente.</p>"
        ))

# ----------------------------- Arranque UI -----------------------------------
try:
    import ipywidgets as W
    from IPython.display import display, HTML, clear_output
    use_widgets = True
except Exception:
    from IPython.display import display, HTML, clear_output
    use_widgets = False

equipos = sorted(teams_long['team'].dropna().unique().tolist())

if use_widgets:
    ddA = W.Dropdown(options=equipos, description='Equipo A:', value=equipos[0] if equipos else None, layout=W.Layout(width='350px'))
    ddB = W.Dropdown(options=equipos, description='Equipo B:', value=(equipos[1] if len(equipos)>1 else (equipos[0] if equipos else None)), layout=W.Layout(width='350px'))
    btn = W.Button(description="Mostrar", button_style='primary')
    out = W.Output()

    def _click(_):
        with out:
            out.clear_output(wait=True)
            render_compare(ddA.value, ddB.value)

    btn.on_click(_click)
    ctrls = W.HBox([ddA, ddB, btn])
    display(ctrls, out)
    _click(None)
else:
    teamA = equipos[0] if equipos else None
    teamB = equipos[1] if len(equipos)>1 else (equipos[0] if equipos else None)
    render_compare(teamA, teamB)
# =============================================================================



Output()

**ENTRENAMIENTO DEL MODELO PARA PREDICCIONES**

In [77]:
# ======================= MODELADO: FEATURES + ENTRENAMIENTO =======================
import numpy as np
import pandas as pd
from datetime import timedelta

# Modelos sklearn (nativos, sin dependencias extra)
from sklearn.linear_model import PoissonRegressor, LogisticRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, log_loss, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# -------------------------------------------------------
# Utilidades: seguridad con columnas y tipos
# -------------------------------------------------------
def _to_datetime_safe(s):
    return pd.to_datetime(s, errors='coerce')

def _num(s):
    return pd.to_numeric(s, errors='coerce')

def _has(df, cols):
    return all(c in df.columns for c in cols)

# -------------------------------------------------------
# 1) Preparación base: teams_long limpio y ordenado por fecha
#     Espera columnas: ['date','team','opponent','side','goals','opp_goals', ...]
# -------------------------------------------------------
TL = teams_long.copy()
if 'date' in TL.columns:
    TL['date'] = _to_datetime_safe(TL['date'])
else:
    raise SystemExit("teams_long debe tener columna 'date'.")

for c in ['goals','opp_goals','shots','shots_ot','corners','pens','fouls','yellows','reds']:
    if c in TL.columns:
        TL[c] = _num(TL[c])

# -------------------------------------------------------
# 2) Features de forma por equipo (PRE-PARTIDO)
#    - rolling medias últimas N (separadas por casa/fuera y global)
#    - EWM por días (half_life_days)
#    - puntos últimos N
#    - días de descanso
# -------------------------------------------------------
def _points(gf, ga):
    # 3 victoria, 1 empate, 0 derrota
    if pd.isna(gf) or pd.isna(ga): return np.nan
    return 3 if gf > ga else (1 if gf == ga else 0)

def compute_form_features(df_long, window=5, half_life_days=60):
    df = df_long.sort_values(['team','date']).copy()
    # días desde el partido previo (descanso)
    df['rest_days'] = df.groupby('team')['date'].diff().dt.days

    # puntos del partido
    df['points'] = [ _points(g, og) for g, og in zip(df.get('goals'), df.get('opp_goals')) ]

    # helper rolling
    def roll_mean(g, w=window):
        return g.rolling(w, min_periods=max(2, int(w/2))).mean()

    # Por equipo: global
    feats = []
    grp = df.groupby('team', group_keys=False)
    for team, g in grp:
        g = g.copy()

        # rolling global
        for col in ['goals','opp_goals','shots','shots_ot','corners','pens','fouls','yellows','reds','points']:
            if col in g.columns:
                g[f'roll_{col}_w{window}'] = roll_mean(g[col])

        # rolling por casa/fuera
        for side_val in ['home','away']:
            mask = (g['side']==side_val)
            for col in ['goals','opp_goals','shots','shots_ot','corners','pens','points']:
                if col in g.columns:
                    g.loc[mask, f'roll_{col}_{side_val}_w{window}'] = roll_mean(g.loc[mask, col])

        # EWM por *días*: exponente decae con la diferencia temporal
        # Construimos pesos ewm ~ exp(-delta_days/half_life_days)
        if half_life_days is not None and half_life_days > 0:
            dates = g['date'].values
            # para cada fila i, calculamos una media ponderada de valores previos
            for col in ['goals','opp_goals','shots','shots_ot','corners','pens','points']:
                if col not in g.columns: 
                    continue
                vals = g[col].values.astype(float)
                ewm_vals = np.full(len(g), np.nan, dtype=float)
                for i in range(len(g)):
                    # solo datos anteriores
                    mask_prev = np.arange(len(g)) < i
                    if not mask_prev.any(): 
                        continue
                    dt_days = (dates[i] - dates[mask_prev]).astype('timedelta64[D]').astype(float)
                    # pesos: más recientes pesan más
                    w = np.exp(-dt_days / half_life_days)
                    x = vals[mask_prev]
                    good = np.isfinite(x) & np.isfinite(w)
                    if good.any():
                        ewm_vals[i] = np.average(x[good], weights=w[good])
                g[f'ewm_{col}_hl{half_life_days}'] = ewm_vals

        feats.append(g)

    out = pd.concat(feats, ignore_index=True)
    # IMPORTANTÍSIMO: desfase 1 (shift) para que las features sean PRE-partido
    # (la fila i usa solo info estrictamente anterior)
    sort_cols = ['team','date']
    out = out.sort_values(sort_cols)
    feat_cols = [c for c in out.columns if c.startswith('roll_') or c.startswith('ewm_') or c=='rest_days']
    out[feat_cols] = out.groupby('team')[feat_cols].shift(1)

    return out

form_df = compute_form_features(TL, window=5, half_life_days=60)

# -------------------------------------------------------
# 3) Dataset de PARTIDOS (home vs away) con features previas
#    - Unimos las features del local y del visitante
#    - Generamos diferencias (home - away) como variables finales
# -------------------------------------------------------
# Partidos únicos con home/away (usamos matches_features si existe)
if 'matches_features' in globals() and isinstance(matches_features, pd.DataFrame) and not matches_features.empty:
    MF = matches_features.copy()
    if 'date' in MF.columns: MF['date'] = _to_datetime_safe(MF['date'])
    # columnas esperadas
    name_home = 'h_team' if 'h_team' in MF.columns else ('home_team' if 'home_team' in MF.columns else None)
    name_away = 'a_team' if 'a_team' in MF.columns else ('away_team' if 'away_team' in MF.columns else None)
    name_hg   = 'home_goals' if 'home_goals' in MF.columns else ('h_goals' if 'h_goals' in MF.columns else None)
    name_ag   = 'away_goals' if 'away_goals' in MF.columns else ('a_goals' if 'a_goals' in MF.columns else None)
    base_cols = [c for c in ['match_id','date', name_home, name_away, name_hg, name_ag] if c]
    MF = MF[base_cols].dropna()
    MF = MF.rename(columns={name_home:'home_team', name_away:'away_team', name_hg:'home_goals', name_ag:'away_goals'})
else:
    # reconstruimos desde teams_long
    # Asumimos que cada match aparece dos veces en TL (home y away) — si no hay match_id, generamos por fecha + emparejamiento
    tmp = TL.copy()
    if 'match_id' not in tmp.columns:
        # construir pseudo-id por fecha + pares de equipos
        tmp['match_id'] = (tmp['date'].astype(str) + '_' + tmp[['team','opponent']].astype(str).agg('|'.join, axis=1)).str.replace(' ', '_')
    # pivot: sacar por match local/visitante y marcador
    # Primero identificar home rows
    home_rows = tmp[tmp['side']=='home'][['match_id','date','team','opponent','goals','opp_goals']].rename(
        columns={'team':'home_team','opponent':'away_team','goals':'home_goals','opp_goals':'away_goals'}
    )
    away_rows = tmp[tmp['side']=='away'][['match_id','date','team','opponent','goals','opp_goals']].rename(
        columns={'team':'away_team2','opponent':'home_team2','goals':'away_goals2','opp_goals':'home_goals2'}
    )
    MF = pd.merge(home_rows, away_rows[['match_id','away_team2','home_team2','away_goals2','home_goals2']],
                  on='match_id', how='left')
    # preferimos las columnas del bloque home_rows; si están vacías, usar las otras
    MF['away_team'] = np.where(MF['away_team'].notna(), MF['away_team'], MF['away_team2'])
    MF['home_team'] = np.where(MF['home_team'].notna(), MF['home_team'], MF['home_team2'])
    MF['away_goals'] = np.where(MF['away_goals'].notna(), MF['away_goals'], MF['away_goals2'])
    MF['home_goals'] = np.where(MF['home_goals'].notna(), MF['home_goals'], MF['home_goals2'])
    MF = MF[['match_id','date','home_team','away_team','home_goals','away_goals']].drop_duplicates()

# Features PRE-partido del local y visitante
FE = form_df.copy()
FE_cols = ['team','date','side','rest_days'] + [c for c in FE.columns if c.startswith('roll_') or c.startswith('ewm_')]
FE = FE[FE_cols]

home_feat = FE[FE['side']=='home'].drop(columns=['side']).rename(columns=lambda c: f'home__{c}' if c not in ['date'] else c)
home_feat = home_feat.rename(columns={'home__team':'home_team'})
away_feat = FE[FE['side']=='away'].drop(columns=['side']).rename(columns=lambda c: f'away__{c}' if c not in ['date'] else c)
away_feat = away_feat.rename(columns={'away__team':'away_team'})

# Join por (equipo, fecha) tomando la fila anterior más cercana <= fecha del partido (merge_asof)
MF = MF.sort_values('date')
home_feat = home_feat.sort_values('date')
away_feat = away_feat.sort_values('date')

MF = pd.merge_asof(MF.sort_values('date'),
                   home_feat.sort_values('date'),
                   left_on='date', right_on='date',
                   left_by='home_team', right_by='home_team',
                   direction='backward')

MF = pd.merge_asof(MF.sort_values('date'),
                   away_feat.sort_values('date'),
                   left_on='date', right_on='date',
                   left_by='away_team', right_by='away_team',
                   direction='backward')

# Generar features “diferencia” (home - away) para simplificar
def difference_cols(df):
    diffs = {}
    for c in df.columns:
        if c.startswith('home__'):
            peer = 'away__' + c[len('home__'):]
            if peer in df.columns:
                diffs[f'diff__{c[len("home__"):]}'] = df[c] - df[peer]
    return pd.DataFrame(diffs)

DIFF = difference_cols(MF)
DS = pd.concat([MF, DIFF], axis=1)

# -------------------------------------------------------
# 4) Conjuntos de entrenamiento (time split)
#    (Primero creamos los targets de clasificación; luego split)
# -------------------------------------------------------
DS = DS.dropna(subset=['home_goals','away_goals'])  # objetivos básicos
DS = DS.sort_values('date').reset_index(drop=True)

# Clasificación 1X2: objetivo "home_win" y "draw"
DS['home_win'] = (DS['home_goals'] > DS['away_goals']).astype(int)
DS['draw']     = (DS['home_goals'] == DS['away_goals']).astype(int)

# indices de split temporal (último 20% validación)
cut = int(len(DS)*0.8) if len(DS) > 15 else max(5, int(len(DS)*0.7))
train = DS.iloc[:cut].copy()
valid = DS.iloc[cut:].copy()

# -------------------------------------------------------
# 5) Definición de TARGETS y FEATURES
# -------------------------------------------------------
# Objetivos posibles según disponibilidad
regression_targets = []
for t in ['home_goals','away_goals','home_shots','away_shots','home_shots_ot','away_shots_ot','home_corners','away_corners']:
    if t in DS.columns:
        regression_targets.append(t)

# Conjunto de features: priorizamos diferencias; si faltan, usamos vectores home/away
diff_feats = [c for c in DS.columns if c.startswith('diff__')]
home_feats = [c for c in DS.columns if c.startswith('home__')]
away_feats = [c for c in DS.columns if c.startswith('away__')]

X_cols_primary = diff_feats if len(diff_feats) >= 8 else (diff_feats + home_feats + away_feats)
X_cols_primary = [c for c in X_cols_primary if c not in ['home__team','away__team']]

def _split_Xy(cols, y_name):
    X_tr = train[cols].copy()
    y_tr = train[y_name].copy()
    X_va = valid[cols].copy()
    y_va = valid[y_name].copy()
    # rellenar NaNs con medianas de train (seguro)
    med = X_tr.median(numeric_only=True)
    X_tr = X_tr.fillna(med)
    X_va = X_va.fillna(med)
    return X_tr, y_tr, X_va, y_va

# -------------------------------------------------------
# 6) ENTRENAMIENTO
#    - Poisson para goles (mejor para contajes)
#    - Ridge para tiros/corners (rápido y estable)
#    - Logística (1X2). Binarios (home_win y draw) y luego normalizamos.
# -------------------------------------------------------
models = {}

# --- 6.1 Poisson: home_goals & away_goals
for tgt in ['home_goals','away_goals']:
    if tgt in regression_targets:
        X_tr, y_tr, X_va, y_va = _split_Xy(X_cols_primary, tgt)
        pr = Pipeline([
            ('sc', StandardScaler(with_mean=False)),
            ('po', PoissonRegressor(alpha=1.0, max_iter=1000))
        ])
        pr.fit(X_tr, y_tr)
        pred = pr.predict(X_va).clip(min=0)
        models[f'poisson_{tgt}'] = pr
        print(f"[Poisson {tgt}] MAE={mean_absolute_error(y_va, pred):.3f}  RMSE={(mean_squared_error(y_va, pred)**0.5):.3f}  R2={r2_score(y_va, pred):.3f}")

# --- 6.2 Regresión (Ridge) para otras estadísticas (si existen)
for tgt in ['home_shots','away_shots','home_shots_ot','away_shots_ot','home_corners','away_corners']:
    if tgt in regression_targets:
        X_tr, y_tr, X_va, y_va = _split_Xy(X_cols_primary, tgt)
        rr = Pipeline([
            ('sc', StandardScaler(with_mean=False)),
            ('rg', Ridge(alpha=1.0, random_state=42))
        ])
        rr.fit(X_tr, y_tr)
        pred = rr.predict(X_va)
        models[f'ridge_{tgt}'] = rr
        print(f"[Ridge {tgt}] MAE={mean_absolute_error(y_va, pred):.3f}  RMSE={(mean_squared_error(y_va, pred)**0.5):.3f}  R2={r2_score(y_va, pred):.3f}")

# --- 6.3 Clasificación 1X2 (binario home_win y binario draw)
for tgt in ['home_win','draw']:
    if tgt in DS.columns:
        # Si hay muy pocas muestras positivas o negativas, evitamos fallos
        if train[tgt].nunique() < 2:
            print(f"[LogReg {tgt}] Saltado: clase única en train.")
            continue
        X_tr, y_tr, X_va, y_va = _split_Xy(X_cols_primary, tgt)
        clf = Pipeline([
            ('sc', StandardScaler(with_mean=False)),
            ('lr', LogisticRegression(max_iter=1000, class_weight='balanced'))
        ])
        clf.fit(X_tr, y_tr)
        p = clf.predict_proba(X_va)[:,1]
        pred = (p >= 0.5).astype(int)
        print(f"[LogReg {tgt}] LogLoss={log_loss(y_va, np.clip(p,1e-6,1-1e-6)):.3f}  Acc={accuracy_score(y_va, pred):.3f}")
        models[f'logit_{tgt}'] = clf

print("✅ Entrenamiento completado (parche aplicado).")

# -------------------------------------------------------
# 7) INTERFAZ DE PREDICCIÓN: partido puntual home vs away
#     - Usa la última forma disponible antes de 'as_of' (por defecto, máxima fecha de DS + 1 día)
# -------------------------------------------------------
def _latest_team_row(FE_all, team, side, cutoff_date):
    t = FE_all[(FE_all['date']<=cutoff_date) & (FE_all['side']==side) & (FE_all['team']==team)].sort_values('date')
    if t.empty:
        return None
    row = t.iloc[-1].copy()
    # renombrar a home__/away__
    prefix = f'{side}__'
    row = row.drop(labels=['side'])
    row.index = [prefix + idx if idx not in ['date','team'] else idx for idx in row.index]
    if side=='home':
        row = row.rename({'team':'home_team'})
    else:
        row = row.rename({'team':'away_team'})
    return row

def predict_match(home_team, away_team, as_of=None):
    """
    Devuelve dict con predicciones:
      - goles esperados (home/away)
      - tiros/tiros OT/corners esperados (si hubo modelo)
      - probabilidades 1X2 (normalizadas)
    """
    if as_of is None:
        as_of = DS['date'].max() + timedelta(days=1)

    # construir fila de features igual que entrenamiento
    home_row = _latest_team_row(form_df.copy()[['team','date','side','rest_days'] + 
                    [c for c in form_df.columns if c.startswith('roll_') or c.startswith('ewm_')]], 
                    home_team, 'home', as_of)
    away_row = _latest_team_row(form_df.copy()[['team','date','side','rest_days'] + 
                    [c for c in form_df.columns if c.startswith('roll_') or c.startswith('ewm_')]], 
                    away_team, 'away', as_of)
    if home_row is None or away_row is None:
        return {"error":"No hay forma histórica suficiente para uno de los equipos."}

    row = pd.concat([home_row, away_row], axis=0).to_frame().T
    # generar diff__
    diffs = {}
    for c in row.columns:
        if c.startswith('home__'):
            peer = 'away__' + c[len('home__'):]
            if peer in row.columns:
                diffs[f'diff__{c[len("home__"):]}'] = row[c].values[0] - row[peer].values[0]
    row_diff = pd.DataFrame(diffs, index=[0])
    row_full = pd.concat([row, row_diff], axis=1)

    # misma selección de columnas que en entrenamiento
    X_cols = [c for c in X_cols_primary if c in row_full.columns]
    X = row_full[X_cols].copy()
    # rellenar NaNs con medianas de train (estimamos con medianas de train)
    med = train[X_cols].median(numeric_only=True)
    X = X.fillna(med)

    out = {"home_team": home_team, "away_team": away_team, "as_of": pd.to_datetime(as_of).strftime("%Y-%m-%d")}
    # goles
    for tgt in ['home_goals','away_goals']:
        key = f'poisson_{tgt}'
        if key in models:
            yhat = float(models[key].predict(X).clip(min=0)[0])
            out[f'pred_{tgt}'] = yhat

    # otras stats
    for tgt in ['home_shots','away_shots','home_shots_ot','away_shots_ot','home_corners','away_corners']:
        key = f'ridge_{tgt}'
        if key in models:
            out[f'pred_{tgt}'] = float(models[key].predict(X)[0])

    # 1X2
    pH = models['logit_home_win'].predict_proba(X)[:,1] if 'logit_home_win' in models else np.array([np.nan])
    pD = models['logit_draw'].predict_proba(X)[:,1]     if 'logit_draw' in models else np.array([np.nan])
    pH = float(pH[0]) if np.isfinite(pH[0]) else np.nan
    pD = float(pD[0]) if np.isfinite(pD[0]) else np.nan
    if np.isfinite(pH) and np.isfinite(pD):
        pA = max(0.0, 1.0 - pH - pD)
        # renormaliza si hay pequeños desajustes
        s = pH + pD + pA
        if s > 0: pH, pD, pA = pH/s, pD/s, pA/s
        out['prob_1X2'] = {"home": round(pH,3), "draw": round(pD,3), "away": round(pA,3)}
    else:
        out['prob_1X2'] = {"home": np.nan, "draw": np.nan, "away": np.nan}

    # totales derivados
    if 'pred_home_goals' in out and 'pred_away_goals' in out:
        out['pred_total_goals'] = out['pred_home_goals'] + out['pred_away_goals']

    return out

# ---------------- Ejemplo de uso ----------------
# ej = predict_match("Real Madrid", "Barcelona")
# ej
# ------------------------------------------------------------------------------


[Poisson home_goals] MAE=1.175  RMSE=1.647  R2=-0.150
[Poisson away_goals] MAE=0.954  RMSE=1.231  R2=-0.018
[LogReg home_win] LogLoss=0.620  Acc=0.500
[LogReg draw] LogLoss=0.418  Acc=0.900
✅ Entrenamiento completado (parche aplicado).


# ¿Qué significan estas métricas?

A continuación se explica, de forma breve, qué mide cada métrica que imprime el notebook y cómo interpretarla en el contexto de fútbol.

---

## 1) Regresión (goles, tiros, corners…)

Se usan **MAE**, **RMSE** y **R²**.

- **MAE (Mean Absolute Error)**  
  Promedio del error absoluto.  
    - **Más bajo = mejor.**  
  - En goles (rango típico 0–4), un MAE ≈ 0.6–0.9 suele ser razonable; >1.0 indica que todavía falta señal.

- **RMSE (Root Mean Squared Error)**  
  Raíz del error cuadrático medio. Penaliza más los errores grandes.  
    - **Más bajo = mejor.** Siempre ≥ MAE.

- **R² (Coeficiente de determinación)**  
  Proporción de varianza explicada por el modelo frente a un modelo tonto (la media).  
   - **1.0 = perfecto; 0.0 = igual que predecir la media; negativo = peor que la media.**  
  - En fútbol, el **R² en goles** suele ser bajo por la alta aleatoriedad del deporte.

---


## 2) Cómo leer los resultados actuales

- **Poisson home_goals** → `MAE ≈ 1.31`, `RMSE ≈ 1.77`, `R² ≈ -0.33`  
  - Error medio de ~1.3 goles: **alto** para un rango 0–4.  
  - R² negativo: **peor que predecir la media**. Necesita más features (forma, xG, H2H, ranking…).

- **Poisson away_goals** → `MAE ≈ 0.92`, `RMSE ≈ 1.19`, `R² ≈ -0.01`  
  - Algo mejor, pero **R² ≈ 0** indica que apenas mejora a la media.

- **LogReg home_win** → `LogLoss ≈ 0.642`, `Acc ≈ 0.60`  
  - 60% de acierto: **aceptable** para un baseline inicial, aunque mejorable.

- **LogReg draw** → `LogLoss ≈ 0.422`, `Acc ≈ 0.70`  
  - 70% puede estar **inflado** si hay **pocos empates** (desbalance). Conviene mirar F1 para la clase “empate”.

---


In [78]:
# ================== CONSTRUIR teams_long DESDE laliga_matches_features_2025.csv ==================
import os, re, unicodedata
import pandas as pd
import numpy as np

def _to_datetime_safe(s): return pd.to_datetime(s, errors='coerce')

def load_first_existing(paths):
    for p in paths:
        if p and os.path.exists(p):
            return pd.read_csv(p)
    return pd.DataFrame()

def deaccent(s):
    if pd.isna(s): return s
    import unicodedata, re
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"\b(cf|fc|sd|sad|ud|cd)\b", "", s)
    s = s.replace(".", " ").replace(",", " ").replace("-", " ")
    s = s.replace("atletico de madrid","atletico madrid")
    s = s.replace("real betis balompie","real betis")
    s = s.replace("athletic club","athletic bilbao")
    s = s.replace("ud las palmas","las palmas")
    return re.sub(r"\s+"," ", s).strip()

def pick_col(df, *patterns):
    pats = [p.lower() for p in patterns]
    for c in df.columns:
        lc = str(c).lower()
        if any(p in lc for p in pats):
            return c
    return None

# 1) Carga el CSV de partidos con features
df = load_first_existing([
    "laliga_matches_with_fbref_2025.csv",   # si lo tienes
    "laliga_matches_features_2025.csv"      # creado en tu Celda 6
])
if df.empty:
    raise SystemExit("No encuentro 'laliga_matches_features_2025.csv' (ni el with_fbref). Genera ese CSV y reintenta.")

# 2) Normaliza fecha y detecta columnas clave
if 'date' in df.columns:
    df['date'] = _to_datetime_safe(df['date'])

h_team = pick_col(df, "h_team","home_team")
a_team = pick_col(df, "a_team","away_team")
h_goals = pick_col(df, "h_goals","home_goals")
a_goals = pick_col(df, "a_goals","away_goals")
if not all([h_team, a_team, h_goals, a_goals]):
    raise SystemExit("Faltan columnas de equipos/goles (home/away). Revisa tu CSV de matches_features.")

home_sot  = pick_col(df, "home_shots_on_target","home_sot","home_shots_ot")
away_sot  = pick_col(df, "away_shots_on_target","away_sot","away_shots_ot")
home_pens = pick_col(df, "home_pen","home_pens","home_penalty","home_penalties")
away_pens = pick_col(df, "away_pen","away_pens","away_penalty","away_penalties")
home_corn = pick_col(df, "home_corners","home_corner","home_corners_count")
away_corn = pick_col(df, "away_corners","away_corner","away_corners_count")

# 3) Construye formato largo (home / away)
home_cols = {'team':h_team, 'opponent':a_team, 'goals':h_goals}
away_cols = {'team':a_team, 'opponent':h_team, 'goals':a_goals}
if 'home_shots' in df.columns: home_cols['shots'] = 'home_shots'
if 'away_shots' in df.columns: away_cols['shots'] = 'away_shots'
if home_sot:  home_cols['shots_ot'] = home_sot
if away_sot:  away_cols['shots_ot'] = away_sot
if home_pens: home_cols['pens']     = home_pens
if away_pens: away_cols['pens']     = away_pens
if home_corn: home_cols['corners']  = home_corn
if away_corn: away_cols['corners']  = away_corn

def make_side(df_in, mapper, side):
    cols = {k:v for k,v in mapper.items() if v in df_in.columns}
    base = (['date'] if 'date' in df_in.columns else []) + list(cols.values())
    out = df_in[base].copy()
    out = out.rename(columns={v:k for k,v in cols.items()})
    out['side'] = side
    return out

df_home = make_side(df, home_cols, 'home')
df_away = make_side(df, away_cols, 'away')

teams_long = pd.concat([df_home, df_away], ignore_index=True, sort=False)
teams_long['team_norm'] = teams_long['team'].map(deaccent)
teams_long['opp_norm']  = teams_long['opponent'].map(deaccent)

print("✅ teams_long creado con", len(teams_long), "filas y columnas:", list(teams_long.columns))
print(teams_long.head(3))


✅ teams_long creado con 96 filas y columnas: ['date', 'team', 'opponent', 'goals', 'shots', 'shots_ot', 'pens', 'corners', 'side', 'team_norm', 'opp_norm']
                 date        team        opponent  goals  shots  shots_ot  \
0 2025-08-15 17:00:00      Girona  Rayo Vallecano      1    7.0       2.0   
1 2025-08-15 19:30:00  Villarreal     Real Oviedo      2   25.0      10.0   
2 2025-08-16 17:30:00    Mallorca       Barcelona      0    4.0       1.0   

   pens  corners  side   team_norm        opp_norm  
0   0.0      1.0  home      girona  rayo vallecano  
1   0.0      5.0  home  villarreal     real oviedo  
2   0.0      1.0  home    mallorca       barcelona  


In [79]:
# =======================================
# EVALUACIÓN COMPLETA (robusta, 1 sola celda)
# - Construye teams_long si no existe (desde laliga_matches_features_2025.csv)
# - Asegura opp_goals en teams_long
# - Construye MF (home_team, away_team, marcador)
# - Calcula forma (rolling/EWM) pre-partido
# - Añade H2H
# - Construye DS + X_cols_primary
# - Walk-forward con imputación segura (sin NaNs)
# =======================================
import os
import numpy as np
import pandas as pd
from collections import defaultdict

# sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PoissonRegressor, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, log_loss, accuracy_score

# ----------------- utilidades -----------------
def _to_datetime_safe(s): return pd.to_datetime(s, errors='coerce')
def _num(s): return pd.to_numeric(s, errors='coerce')

def _print(msg):
    print(f"[INFO] {msg}")

def load_first_existing(paths):
    for p in paths:
        if p and os.path.exists(p):
            return pd.read_csv(p)
    return pd.DataFrame()

def pick_col(df, *patterns):
    pats = [p.lower() for p in patterns]
    for c in df.columns:
        lc = str(c).lower()
        if any(p in lc for p in pats):
            return c
    return None

# ------------- 0) teams_long: construir si no existe -------------
need_build_teams_long = ('teams_long' not in globals()) or (not isinstance(globals().get('teams_long'), pd.DataFrame)) or globals().get('teams_long').empty

if need_build_teams_long:
    df = load_first_existing([
        "laliga_matches_with_fbref_2025.csv",
        "laliga_matches_features_2025.csv"
    ])
    if df.empty:
        _print("No encuentro CSV de partidos. Si ya tienes teams_long en memoria, ignora este mensaje; si no, crea 'laliga_matches_features_2025.csv'.")
        teams_long = pd.DataFrame()
    else:
        if 'date' in df.columns:
            df['date'] = _to_datetime_safe(df['date'])
        h_team = pick_col(df, "h_team","home_team")
        a_team = pick_col(df, "a_team","away_team")
        h_goals = pick_col(df, "h_goals","home_goals")
        a_goals = pick_col(df, "a_goals","away_goals")
        if not all([h_team, a_team, h_goals, a_goals]):
            _print("El CSV no contiene columnas estándar de equipos/goles; intentaré construir con lo disponible.")
        # columnas extra si existen
        home_sot  = pick_col(df, "home_shots_on_target","home_sot","home_shots_ot")
        away_sot  = pick_col(df, "away_shots_on_target","away_sot","away_shots_ot")
        home_pens = pick_col(df, "home_pen","home_pens","home_penalty","home_penalties")
        away_pens = pick_col(df, "away_pen","away_pens","away_penalty","away_penalties")
        home_corn = pick_col(df, "home_corners","home_corner","home_corners_count")
        away_corn = pick_col(df, "away_corners","away_corner","away_corners_count")

        # lado local
        home_cols = {'team':h_team, 'opponent':a_team, 'goals':h_goals}
        away_cols = {'team':a_team, 'opponent':h_team, 'goals':a_goals}
        if 'home_shots' in df.columns: home_cols['shots'] = 'home_shots'
        if 'away_shots' in df.columns: away_cols['shots'] = 'away_shots'
        if home_sot:  home_cols['shots_ot'] = home_sot
        if away_sot:  away_cols['shots_ot'] = away_sot
        if home_pens: home_cols['pens']     = home_pens
        if away_pens: away_cols['pens']     = away_pens
        if home_corn: home_cols['corners']  = home_corn
        if away_corn: away_cols['corners']  = away_corn

        def make_side(df_in, mapper, side):
            cols = {k:v for k,v in mapper.items() if v in df_in.columns}
            base = (['date'] if 'date' in df_in.columns else []) + list(cols.values())
            out = df_in[base].copy()
            out = out.rename(columns={v:k for k,v in cols.items()})
            out['side'] = side
            return out

        df_home = make_side(df, home_cols, 'home')
        df_away = make_side(df, away_cols, 'away')
        teams_long = pd.concat([df_home, df_away], ignore_index=True, sort=False)
        _print(f"teams_long creado con {len(teams_long)} filas.")

else:
    teams_long = globals().get('teams_long').copy()
    if 'date' in teams_long.columns:
        teams_long['date'] = _to_datetime_safe(teams_long['date'])
    _print("Usando teams_long del entorno actual.")

# ------------- 1) asegurar opp_goals en teams_long -------------
if not teams_long.empty:
    if 'opp_goals' not in teams_long.columns or teams_long['opp_goals'].isna().all():
        base = teams_long[['date','team','opponent','goals']].copy()
        base['date'] = _to_datetime_safe(base['date'])
        mirror = base.rename(columns={'team':'opponent','opponent':'team','goals':'opp_goals'})
        teams_long = teams_long.merge(mirror, on=['date','team','opponent'], how='left')
        if 'opp_goals_x' in teams_long.columns and 'opp_goals_y' in teams_long.columns:
            teams_long['opp_goals'] = teams_long['opp_goals_x'].combine_first(teams_long['opp_goals_y'])
            teams_long = teams_long.drop(columns=['opp_goals_x','opp_goals_y'])
        _print("Asegurado teams_long['opp_goals'].")

# ------------- 2) MF (partidos únicos home vs away) -------------
def build_MF_from_sources(teams_long, matches_features=None):
    if matches_features is not None and isinstance(matches_features, pd.DataFrame) and not matches_features.empty:
        MF = matches_features.copy()
        MF['date'] = _to_datetime_safe(MF['date'])
        name_home = 'h_team' if 'h_team' in MF.columns else ('home_team' if 'home_team' in MF.columns else None)
        name_away = 'a_team' if 'a_team' in MF.columns else ('away_team' if 'away_team' in MF.columns else None)
        name_hg   = 'home_goals' if 'home_goals' in MF.columns else ('h_goals' if 'h_goals' in MF.columns else None)
        name_ag   = 'away_goals' if 'away_goals' in MF.columns else ('a_goals' if 'a_goals' in MF.columns else None)
        base_cols = [c for c in ['match_id','date',name_home,name_away,name_hg,name_ag] if c]
        MF = MF[base_cols].dropna().rename(columns={
            name_home:'home_team', name_away:'away_team', name_hg:'home_goals', name_ag:'away_goals'
        })
        return MF.sort_values('date').reset_index(drop=True)

    TL = teams_long.copy()
    TL['date'] = _to_datetime_safe(TL['date'])
    if TL.empty:
        return pd.DataFrame(columns=['match_id','date','home_team','away_team','home_goals','away_goals'])

    if 'match_id' not in TL.columns:
        TL['match_id'] = (TL['date'].astype(str)+'_'+TL[['team','opponent']].astype(str).agg('|'.join, axis=1)).str.replace(' ','_', regex=False)

    # Home rows
    home_cols = ['match_id','date','team','opponent','goals']
    if 'opp_goals' in TL.columns: home_cols.append('opp_goals')
    home_rows = TL[TL['side']=='home'][home_cols].rename(
        columns={'team':'home_team','opponent':'away_team','goals':'home_goals','opp_goals':'away_goals'}
    )

    # Away rows
    away_cols = ['match_id','team','opponent','goals']
    if 'opp_goals' in TL.columns: away_cols.append('opp_goals')
    away_rows = TL[TL['side']=='away'][away_cols].rename(
        columns={'team':'away_team2','opponent':'home_team2','goals':'away_goals2','opp_goals':'home_goals2'}
    )

    # Merge con solo columnas existentes
    merge_right_cols = ['match_id'] + [c for c in ['away_team2','home_team2','away_goals2','home_goals2'] if c in away_rows.columns]
    MF = pd.merge(home_rows, away_rows[merge_right_cols], on='match_id', how='left')

    # Completar equipos
    if 'away_team' not in MF.columns and 'away_team2' in MF.columns:
        MF['away_team'] = MF['away_team2']
    else:
        if 'away_team' in MF.columns and 'away_team2' in MF.columns:
            MF['away_team'] = np.where(MF['away_team'].notna(), MF['away_team'], MF['away_team2'])
    if 'home_team' not in MF.columns and 'home_team2' in MF.columns:
        MF['home_team'] = MF['home_team2']
    else:
        if 'home_team' in MF.columns and 'home_team2' in MF.columns:
            MF['home_team'] = np.where(MF['home_team'].notna(), MF['home_team'], MF['home_team2'])

    # Completar marcadores
    if 'away_goals' not in MF.columns and 'away_goals2' in MF.columns:
        MF['away_goals'] = MF['away_goals2']
    elif 'away_goals' in MF.columns and 'away_goals2' in MF.columns:
        MF['away_goals'] = np.where(MF['away_goals'].notna(), MF['away_goals'], MF['away_goals2'])

    if 'home_goals' not in MF.columns and 'home_goals2' in MF.columns:
        MF['home_goals'] = MF['home_goals2']
    elif 'home_goals' in MF.columns and 'home_goals2' in MF.columns:
        MF['home_goals'] = np.where(MF['home_goals'].notna(), MF['home_goals'], MF['home_goals2'])

    keep = [c for c in ['match_id','date','home_team','away_team','home_goals','away_goals'] if c in MF.columns]
    MF = MF[keep].drop_duplicates()
    MF['date'] = _to_datetime_safe(MF['date'])
    return MF.sort_values('date').reset_index(drop=True)

# ------------- 3) H2H -------------
def add_h2h_if_missing(MF):
    if MF.empty or any(c.startswith('h2h_') for c in MF.columns):
        return MF
    recs = []
    for _, r in MF.iterrows():
        home, away, date = r['home_team'], r['away_team'], r['date']
        prev = MF[(MF['date']<date) & ( ((MF['home_team']==home)&(MF['away_team']==away)) | ((MF['home_team']==away)&(MF['away_team']==home)) )]
        if prev.empty:
            recs.append({'h2h_count':0,'h2h_home_goals_avg':np.nan,'h2h_away_goals_avg':np.nan,
                         'h2h_home_win_rate':np.nan,'h2h_draw_rate':np.nan,'h2h_away_win_rate':np.nan})
            continue
        hg_list, ag_list = [], []; hW=dW=aW=0
        for _, p in prev.iterrows():
            if p['home_team']==home and p['away_team']==away:
                hg, ag = p['home_goals'], p['away_goals']
            else:
                hg, ag = p['away_goals'], p['home_goals']
            hg_list.append(hg); ag_list.append(ag)
            if hg>ag: hW+=1
            elif hg==ag: dW+=1
            else: aW+=1
        n=len(prev)
        recs.append({'h2h_count':n,'h2h_home_goals_avg':float(np.mean(hg_list)),
                     'h2h_away_goals_avg':float(np.mean(ag_list)),
                     'h2h_home_win_rate':hW/n,'h2h_draw_rate':dW/n,'h2h_away_win_rate':aW/n})
    H2H = pd.DataFrame(recs, index=MF.index)
    return pd.concat([MF.reset_index(drop=True), H2H.reset_index(drop=True)], axis=1)

# ------------- 4) forma (rolling/EWM) PRE-partido -------------
def compute_form_features(df_long, window=5, half_life_days=60):
    df = df_long.copy()
    if df.empty:
        return df
    df['date'] = _to_datetime_safe(df['date'])
    for c in ['goals','opp_goals','shots','shots_ot','corners','pens']:
        if c in df.columns: df[c] = _num(df[c])
    df = df.sort_values(['team','date']).copy()
    df['rest_days'] = df.groupby('team')['date'].diff().dt.days

    def _pts(g, og):
        if pd.isna(g) or pd.isna(og): return np.nan
        return 3 if g>og else (1 if g==og else 0)
    if 'opp_goals' not in df.columns:
        df['opp_goals'] = np.nan
    df['points'] = [_pts(g, og) for g, og in zip(df['goals'], df['opp_goals'])]

    def roll_mean(s, w): return s.rolling(w, min_periods=max(2,int(w/2))).mean()
    feats = []
    for team, g in df.groupby('team', group_keys=False):
        g = g.copy()
        for col in ['goals','opp_goals','shots','shots_ot','corners','pens','points']:
            if col in g.columns: g[f'roll_{col}_w{window}'] = roll_mean(g[col], window)
        for side in ['home','away']:
            if 'side' in g.columns:
                m = (g['side']==side)
                for col in ['goals','opp_goals','shots','shots_ot','corners','pens','points']:
                    if col in g.columns:
                        g.loc[m, f'roll_{col}_{side}_w{window}'] = roll_mean(g.loc[m, col], window)
        # EWM por días
        if half_life_days and half_life_days>0:
            dates = g['date'].values
            for col in ['goals','opp_goals','shots','shots_ot','corners','pens','points']:
                if col not in g.columns: continue
                vals = g[col].values.astype(float)
                ewm_vals = np.full(len(g), np.nan)
                for i in range(len(g)):
                    prev = np.arange(len(g)) < i
                    if not prev.any(): continue
                    dt_days = (dates[i]-dates[prev]).astype('timedelta64[D]').astype(float)
                    w = np.exp(-dt_days/half_life_days)
                    x = vals[prev]
                    good = np.isfinite(x) & np.isfinite(w)
                    if good.any(): ewm_vals[i] = np.average(x[good], weights=w[good])
                g[f'ewm_{col}_hl{half_life_days}'] = ewm_vals
        feats.append(g)
    out = pd.concat(feats, ignore_index=True).sort_values(['team','date'])
    feat_cols = [c for c in out.columns if c.startswith('roll_') or c.startswith('ewm_')] + ['rest_days']
    out[feat_cols] = out.groupby('team')[feat_cols].shift(1)  # PRE-partido
    return out

# ------------- 5) DS (dataset final) -------------
def build_DS(teams_long, matches_features=None, include_h2h=True):
    MF = build_MF_from_sources(teams_long, matches_features if 'matches_features' in globals() else None)
    if MF.empty:
        return pd.DataFrame()

    if include_h2h:
        MF = add_h2h_if_missing(MF)

    FE = compute_form_features(teams_long, window=5, half_life_days=60)
    if FE.empty:
        return pd.DataFrame()

    FE_cols = ['team','date','side','rest_days'] + [c for c in FE.columns if c.startswith('roll_') or c.startswith('ewm_')]
    FE = FE[FE_cols]

    home_feat = FE[FE['side']=='home'].drop(columns=['side']).rename(columns=lambda c: f'home__{c}' if c!='date' else c)
    home_feat = home_feat.rename(columns={'home__team':'home_team'})
    away_feat = FE[FE['side']=='away'].drop(columns=['side']).rename(columns=lambda c: f'away__{c}' if c!='date' else c)
    away_feat = away_feat.rename(columns={'away__team':'away_team'})

    MF = MF.sort_values('date')
    home_feat = home_feat.sort_values('date')
    away_feat = away_feat.sort_values('date')

    MFj = pd.merge_asof(MF, home_feat, on='date', left_by='home_team', right_by='home_team', direction='backward')
    MFj = pd.merge_asof(MFj, away_feat, on='date', left_by='away_team', right_by='away_team', direction='backward')

    # diferencias home-away
    diffs = {}
    for c in MFj.columns:
        if c.startswith('home__'):
            peer = 'away__' + c[len('home__'):]
            if peer in MFj.columns:
                diffs[f'diff__{c[len('home__') :]}'] = MFj[c] - MFj[peer]
    DIFF = pd.DataFrame(diffs, index=MFj.index)

    DS = pd.concat([MFj, DIFF], axis=1)
    if not {'home_goals','away_goals'}.issubset(DS.columns):
        return pd.DataFrame()
    DS = DS.dropna(subset=['home_goals','away_goals']).sort_values('date').reset_index(drop=True)
    DS['home_win'] = (DS['home_goals'] > DS['away_goals']).astype(int)
    DS['draw']     = (DS['home_goals'] == DS['away_goals']).astype(int)
    return DS

# Construir DS
DS = build_DS(teams_long, matches_features=globals().get('matches_features', None), include_h2h=True)

if DS.empty:
    _print("DS está vacío. Revisa que teams_long/matches_features tengan info suficiente. No se ejecutará la evaluación.")
else:
    # ------------- 6) X_cols_primary -------------
    diff_feats = [c for c in DS.columns if c.startswith('diff__')]
    home_feats = [c for c in DS.columns if c.startswith('home__')]
    away_feats = [c for c in DS.columns if c.startswith('away__')]
    h2h_cols   = [c for c in DS.columns if c.startswith('h2h_')]
    X_cols_primary = h2h_cols + [c for c in (diff_feats if len(diff_feats)>=8 else diff_feats+home_feats+away_feats)
                                if c not in ['home__team','away__team']]

    # ------------- 7) Walk-forward robusto -------------
    def _valid_feature_subset(X):
        X = X.replace([np.inf, -np.inf], np.nan)
        cols = [c for c in X.columns if X[c].notna().any()]
        return X[cols], cols

    def _pipe_poisson():
        return Pipeline([
            ('imp', SimpleImputer(strategy='median')),
            ('sc',  StandardScaler(with_mean=False)),
            ('po',  PoissonRegressor(alpha=1.0, max_iter=1000))
        ])

    def _pipe_logit():
        return Pipeline([
            ('imp', SimpleImputer(strategy='median')),
            ('sc',  StandardScaler(with_mean=False)),
            ('lr',  LogisticRegression(max_iter=1000, class_weight='balanced'))
        ])

    results = defaultdict(list)
    n_splits = min(5, max(2, len(DS)//10))  # evita splits imposibles si hay pocos partidos
    tscv = TimeSeriesSplit(n_splits=n_splits)

    ran_any_split = False
    for split, (tr_idx, va_idx) in enumerate(tscv.split(DS)):
        train = DS.iloc[tr_idx].copy()
        valid = DS.iloc[va_idx].copy()
        X_cols = [c for c in X_cols_primary if c in DS.columns]
        if len(X_cols) == 0 or len(train) < 10 or len(valid) < 3:
            continue

        X_tr0 = train[X_cols].copy().replace([np.inf, -np.inf], np.nan)
        X_va0 = valid[X_cols].copy().replace([np.inf, -np.inf], np.nan)
        X_tr, cols_ok = _valid_feature_subset(X_tr0)
        X_va = X_va0[cols_ok].copy()
        if len(cols_ok) < 3:
            continue

        ran_any_split = True

        # Poisson: goles
        for tgt in ['home_goals','away_goals']:
            if tgt not in train.columns: 
                continue
            y_tr, y_va = train[tgt].values, valid[tgt].values
            model = _pipe_poisson()
            model.fit(X_tr, y_tr)
            pred = model.predict(X_va).clip(min=0)
            results[f"{tgt}_MAE"].append(mean_absolute_error(y_va, pred))
            results[f"{tgt}_RMSE"].append(mean_squared_error(y_va, pred)**0.5)

        # Clasificación: home_win / draw
        for tgt in ['home_win','draw']:
            if tgt not in train.columns or train[tgt].nunique() < 2:
                continue
            y_tr, y_va = train[tgt].values, valid[tgt].values
            clf = _pipe_logit()
            clf.fit(X_tr, y_tr)
            p = clf.predict_proba(X_va)[:,1]
            pred = (p >= 0.5).astype(int)
            # proteger logloss ante clases vacías
            try:
                results[f"{tgt}_LogLoss"].append(log_loss(y_va, np.clip(p,1e-6,1-1e-6)))
            except ValueError:
                pass
            results[f"{tgt}_Acc"].append(accuracy_score(y_va, pred))

    if not ran_any_split:
        _print("No se pudo ejecutar ningún split válido (muy pocos partidos o pocas columnas útiles). Añade más datos y reintenta.")
    else:
        print("=== Walk-forward (robusto) ===")
        for k,v in results.items():
            if len(v)>0:
                print(f"{k}: {np.mean(v):.3f} ± {np.std(v):.3f}  (n={len(v)})")
            else:
                print(f"{k}: sin datos")




[INFO] Usando teams_long del entorno actual.
[INFO] Asegurado teams_long['opp_goals'].
=== Walk-forward (robusto) ===
home_goals_MAE: 0.857 ± 0.250  (n=3)
home_goals_RMSE: 1.098 ± 0.414  (n=3)
away_goals_MAE: 0.879 ± 0.041  (n=3)
away_goals_RMSE: 1.063 ± 0.133  (n=3)
home_win_LogLoss: 1.091 ± 0.572  (n=3)
home_win_Acc: 0.407 ± 0.210  (n=3)
draw_LogLoss: 0.868 ± 0.434  (n=3)
draw_Acc: 0.407 ± 0.343  (n=3)


In [80]:
# ==================== COMPAT LAYER: MF/H2H/FORM (ligero y tolerante) ====================
import pandas as pd, numpy as np

def _col(df, candidates):
    """Devuelve el primer nombre de columna que exista en df de entre candidates."""
    return next((c for c in candidates if c in df.columns), None)

def build_MF_from_sources(teams_long, matches_features=None):
    """
    Devuelve una tabla de partidos (MF) con al menos: date, home_team, away_team, (opcional) goles.
    1) Si viene matches_features válido -> lo normaliza y lo devuelve.
    2) Si teams_long está en formato largo con ['team','opponent','side','date'] -> lo compacta a 1 fila/partido.
    """
    # 1) matches_features ya preparado
    MF = matches_features
    if isinstance(MF, pd.DataFrame) and not MF.empty:
        MF = MF.copy()
        # Normalizar nombres
        h = _col(MF, ['home_team','h_team','local','home','team_home'])
        a = _col(MF, ['away_team','a_team','visitante','away','team_away'])
        d = _col(MF, ['date','match_date','utc_date','kickoff'])
        if not (h and a and d):
            # Si no tiene home/away claros, intentamos derivarlo de teams_long
            MF = None
        else:
            MF = MF.rename(columns={h:'home_team', a:'away_team', d:'date'})
            # Normalizar goles si existen
            hg = _col(MF, ['home_goals','h_goals','home_score','goals_home'])
            ag = _col(MF, ['away_goals','a_goals','away_score','goals_away'])
            if hg and hg != 'home_goals': MF = MF.rename(columns={hg:'home_goals'})
            if ag and ag != 'away_goals': MF = MF.rename(columns={ag:'away_goals'})
            MF['date'] = pd.to_datetime(MF['date'], errors='coerce')
            return MF[['date','home_team','away_team'] + ([c for c in ['home_goals','away_goals'] if c in MF.columns])]

    # 2) Derivar desde teams_long (formato largo)
    TL = teams_long.copy()
    if TL.empty: 
        return pd.DataFrame(columns=['date','home_team','away_team'])
    d = _col(TL, ['date','match_date','utc_date'])
    t = _col(TL, ['team','equipo','squad'])
    o = _col(TL, ['opponent','opp','rival','opposition'])
    s = _col(TL, ['side'])
    if not (d and t and o and s):
        # Último recurso: si ya tuviera home/away
        h = _col(TL, ['home_team','h_team','local','home','team_home'])
        a = _col(TL, ['away_team','a_team','visitante','away','team_away'])
        if h and a and d:
            MF = TL[[d,h,a]].drop_duplicates().rename(columns={d:'date', h:'home_team', a:'away_team'})
            MF['date'] = pd.to_datetime(MF['date'], errors='coerce')
            return MF
        return pd.DataFrame(columns=['date','home_team','away_team'])

    TL['date'] = pd.to_datetime(TL[d], errors='coerce')
    # Tomamos solo filas donde el equipo figura como 'home'
    homes = TL[TL[s].astype(str).str.lower().eq('home')].copy()
    if homes.empty:
        # Si no hay 'home', intentamos inferir: si existe indicador 'h_a' ('h'/'a')
        ha = _col(TL, ['h_a'])
        if ha:
            homes = TL[TL[ha].astype(str).str.lower().eq('h')].copy()
    if homes.empty:
        # Fallback: asumimos que la fila representa el partido (sin info de 'side') -> no podemos construir MF robusto
        return pd.DataFrame(columns=['date','home_team','away_team'])

    MF = homes.rename(columns={t:'home_team', o:'away_team', d:'date'})
    # Intentar goles si están
    hg = _col(TL, ['goals_home','home_goals','h_goals','home_score'])
    ag = _col(TL, ['goals_away','away_goals','a_goals','away_score'])
    if hg in homes.columns:
        MF = MF.rename(columns={hg:'home_goals'})
    if ag in homes.columns:
        MF = MF.rename(columns={ag:'away_goals'})
    return MF[['date','home_team','away_team'] + ([c for c in ['home_goals','away_goals'] if c in MF.columns])].drop_duplicates()

def add_h2h_cols(MF):
    """
    Añade columnas h2h_* por emparejamiento (pares de equipos), estático por par (no incremental por fecha).
    - h2h_count
    - h2h_home_goals_avg, h2h_away_goals_avg (si hay goles)
    - h2h_home_win_rate, h2h_draw_rate, h2h_away_win_rate (si hay goles)
    """
    if MF is None or not isinstance(MF, pd.DataFrame) or MF.empty:
        return MF
    df = MF.copy()
    h = _col(df, ['home_team']); a = _col(df, ['away_team']); d = _col(df, ['date','match_date'])
    if not (h and a): 
        return df
    df['__pair__'] = df[[h,a]].apply(lambda r: tuple(sorted([str(r[h]), str(r[a])])), axis=1)

    # Goles/r_resultado (si existen)
    hg = _col(df, ['home_goals','h_goals','home_score'])
    ag = _col(df, ['away_goals','a_goals','away_score'])
    tmp = df.groupby('__pair__', dropna=False).size().rename('h2h_count').to_frame()

    if hg and ag:
        g = df.groupby('__pair__', dropna=False).agg(
            h2h_home_goals_avg = (hg,'mean'),
            h2h_away_goals_avg = (ag,'mean'),
            _home_wins = (hg, lambda s: np.mean(s.values > df.loc[s.index, ag].values)),
            _draws     = (hg, lambda s: np.mean(s.values == df.loc[s.index, ag].values))
        )
        g['_away_wins'] = 1.0 - g['_home_wins'] - g['_draws']
        g = g.rename(columns={'_home_wins':'h2h_home_win_rate','_draws':'h2h_draw_rate','_away_wins':'h2h_away_win_rate'})
        tmp = tmp.join(g, how='left')

    df = df.merge(tmp, left_on='__pair__', right_index=True, how='left')
    return df.drop(columns=['__pair__'])

def compute_form_features(TL, window=5, half_life_days=60):
    """
    Calcula features de 'forma' rolling por equipo y lado (si existe 'side').
    Genera columnas roll_* para stats comunes que encuentre: shots, shots_ot, corners, fouls, goals.
    """
    if TL is None or not isinstance(TL, pd.DataFrame) or TL.empty:
        return pd.DataFrame(columns=['team','date','side'])
    df = TL.copy()
    t = _col(df, ['team','equipo','squad']); d = _col(df, ['date','match_date']); s = _col(df, ['side'])
    if not (t and d):
        return pd.DataFrame(columns=['team','date','side'])
    if s is None:
        df['side'] = 'any'
        s = 'side'
    df['date'] = pd.to_datetime(df[d], errors='coerce')
    df = df[[t,'date',s] + [c for c in df.columns if c not in [t,'date',s]]].sort_values(['team','side','date'] if t=='team' else [t,s,'date'])
    if t != 'team': df = df.rename(columns={t:'team'})

    # Stats candidatas (si no están, se ignoran)
    cand_stats = [
        ('shots',      ['shots']),
        ('shots_ot',   ['shots_ot','sot','shots_on_target']),
        ('corners',    ['corners','corner_shots','corner']),
        ('fouls',      ['fouls']),
        ('goals',      ['goals','gf'])
    ]
    out = []
    for (team, side), grp in df.groupby(['team', s], dropna=False):
        g = grp.copy()
        for base, syns in cand_stats:
            col = _col(g, syns)
            if not col: 
                continue
            x = pd.to_numeric(g[col], errors='coerce')
            g[f'roll_{base}_mean_{window}'] = x.rolling(window, min_periods=1).mean()
            # EWM por fechas (si hay spacing razonable, si no, equivalente a rolling con alpha)
            g[f'ewm_{base}'] = x.ewm(alpha=2.0/max(window,2), adjust=False).mean()
        out.append(g)
    FE = pd.concat(out, ignore_index=True) if out else df.copy()
    return FE

# Nota: build_DS no es estrictamente necesario para tus tarjetas,
# _predict_team_stats ya tiene try/except y caerá a medias recientes si build_DS no está.
# Si más adelante lo necesitas, creamos uno.


In [90]:
# ===== PARCHE: corregir 'module numpy has no attribute math' en la distribución de marcadores =====
import numpy as np, pandas as pd, math
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PoissonRegressor, LogisticRegression

# Reutilizamos helpers ya definidos en tu entorno:
def _valid_feature_subset(X):
    X = X.replace([np.inf, -np.inf], np.nan)
    cols = [c for c in X.columns if X[c].notna().any()]
    return X[cols], cols

def _pipe_poisson():
    return Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('sc',  StandardScaler(with_mean=False)),
        ('po',  PoissonRegressor(alpha=1.0, max_iter=1000))
    ])

def _pipe_logit():
    return Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('sc',  StandardScaler(with_mean=False)),
        ('lr',  LogisticRegression(max_iter=1000, class_weight='balanced'))
    ])

# PMF de Poisson segura (sin np.math)
def _poisson_vec(lam, k_max):
    lam = max(1e-6, float(lam))
    pmf = [math.exp(-lam) * (lam**k) / math.factorial(k) for k in range(k_max+1)]
    s = sum(pmf)
    return np.array([p/s for p in pmf], dtype=float)

def predict_match(home_team, away_team, as_of=None, max_goals=6, top_k=8, verbose=True):
    """
    Entrena con datos <= as_of y predice λ_home/λ_away, prob. 1X2 (logit y Poisson) y top marcadores.
    Requiere que existan en el entorno: teams_long y las funciones build_DS, build_MF_from_sources, add_h2h_cols, compute_form_features.
    """
    # 1) preparar DS si falta
    global DS
    if 'DS' not in globals() or not isinstance(DS, pd.DataFrame) or DS.empty:
        if 'teams_long' not in globals() or not isinstance(teams_long, pd.DataFrame) or teams_long.empty:
            raise SystemExit("Falta teams_long en el entorno.")
        DS_tmp = build_DS(teams_long, matches_features=globals().get('matches_features', None), include_h2h=True)
        if DS_tmp.empty:
            raise SystemExit("No se pudo construir DS. Revisa datos de entrada.")
        DS = DS_tmp.copy()
    DS['date'] = pd.to_datetime(DS['date'], errors='coerce')

    # 2) fecha de corte
    if as_of is None:
        as_of = DS['date'].max() + timedelta(days=1)
    else:
        as_of = pd.to_datetime(as_of, errors='coerce')

    # 3) columnas de features
    diff_feats = [c for c in DS.columns if c.startswith('diff__')]
    home_feats = [c for c in DS.columns if c.startswith('home__')]
    away_feats = [c for c in DS.columns if c.startswith('away__')]
    h2h_cols   = [c for c in DS.columns if c.startswith('h2h_')]
    X_cols_primary = h2h_cols + [c for c in (diff_feats if len(diff_feats)>=8 else diff_feats+home_feats+away_feats)
                                 if c not in ['home__team','away__team']]

    # 4) train hasta as_of
    train = DS[DS['date'] < as_of].copy()
    X_cols = [c for c in X_cols_primary if c in train.columns]
    X_tr0 = train[X_cols].copy().replace([np.inf, -np.inf], np.nan)
    X_tr, cols_ok = _valid_feature_subset(X_tr0)
    if len(cols_ok) < 3:
        raise SystemExit("Muy pocas columnas útiles para entrenar. Añade datos/temporadas.")

    # 5) construir fila de features del partido (forma + h2h previos)
    MF_all = build_MF_from_sources(teams_long, globals().get('matches_features', None))
    MF_all = add_h2h_cols(MF_all)
    MF_all = MF_all[MF_all['date'] < as_of].copy()

    FE_all = compute_form_features(teams_long[teams_long['date'] < as_of], window=5, half_life_days=60)
    FE_cols = ['team','date','side','rest_days'] + [c for c in FE_all.columns if c.startswith('roll_') or c.startswith('ewm_')]
    FE_all = FE_all[FE_cols]

    def _latest_team_row(FE, team, side):
        t = FE[(FE['team']==team) & (FE['side']==side)].sort_values('date')
        if t.empty: return None
        row = t.iloc[-1].drop(labels=['side'])
        row.index = [f'{side}__{ix}' if ix not in ['date','team'] else ix for ix in row.index]
        return row.rename({'team': f'{side}_team'})

    home_row = _latest_team_row(FE_all, home_team, 'home')
    away_row = _latest_team_row(FE_all, away_team, 'away')
    if home_row is None or away_row is None:
        raise SystemExit("No hay forma histórica suficiente para uno de los equipos antes de as_of.")

    row = pd.concat([home_row, away_row]).to_frame().T

    prev = MF_all[( (MF_all['home_team'].isin([home_team, away_team])) &
                    (MF_all['away_team'].isin([home_team, away_team])) )]
    if not prev.empty and any(c.startswith('h2h_') for c in prev.columns):
        last_h2h = prev.sort_values('date').iloc[-1][[c for c in prev.columns if c.startswith('h2h_')]]
        for c in last_h2h.index: row[c] = last_h2h[c]
    else:
        for c in ['h2h_count','h2h_home_goals_avg','h2h_away_goals_avg','h2h_home_win_rate','h2h_draw_rate','h2h_away_win_rate']:
            row[c] = np.nan

    # diffs home-away
    diffs = {}
    for c in row.columns:
        if c.startswith('home__'):
            peer = 'away__' + c[len('home__'):]
            if peer in row.columns:
                diffs[f'diff__{c[len("home__"):]}'] = float(row[c].values[0] - row[peer].values[0])
    row_full = pd.concat([row, pd.DataFrame(diffs, index=[0])], axis=1)

    X = row_full[cols_ok].copy()

    out = {"home_team": home_team, "away_team": away_team, "as_of": pd.to_datetime(as_of).strftime("%Y-%m-%d")}

    # 6) modelos
    pr_home = _pipe_poisson().fit(X_tr, train['home_goals'])
    pr_away = _pipe_poisson().fit(X_tr, train['away_goals'])
    lam_home = float(pr_home.predict(X)[0]); lam_away = float(pr_away.predict(X)[0])
    lam_home = max(0.01, lam_home); lam_away = max(0.01, lam_away)
    out['lambda_home'] = round(lam_home, 3); out['lambda_away'] = round(lam_away, 3)

    pH = pD = np.nan
    if train['home_win'].nunique() >= 2:
        pH = float(_pipe_logit().fit(X_tr, train['home_win']).predict_proba(X)[:,1][0])
    if train['draw'].nunique() >= 2:
        pD = float(_pipe_logit().fit(X_tr, train['draw']).predict_proba(X)[:,1][0])
    if np.isfinite(pH) and np.isfinite(pD):
        pA = max(0.0, 1.0 - pH - pD)
        s = pH + pD + pA
        pH, pD, pA = pH/s, pD/s, pA/s
        out['prob_1X2'] = {'home': round(pH,3), 'draw': round(pD,3), 'away': round(pA,3)}
    else:
        out['prob_1X2'] = {'home': np.nan, 'draw': np.nan, 'away': np.nan}

    # 7) distribución de marcadores (Poisson) — ¡sin np.math!
    gh = np.arange(0, max_goals+1)
    P_h = _poisson_vec(lam_home, max_goals)
    P_a = _poisson_vec(lam_away, max_goals)
    M = np.outer(P_h, P_a)
    M = M / M.sum()  # renormaliza por truncamiento

    p_home = float(np.tril(M, -1).sum())
    p_draw = float(np.trace(M))
    p_away = float(np.triu(M, +1).sum())
    out['prob_1X2_poisson'] = {'home': round(p_home,3), 'draw': round(p_draw,3), 'away': round(p_away,3)}

    score_probs = []
    for i in range(max_goals+1):
        for j in range(max_goals+1):
            score_probs.append((f"{i}-{j}", float(M[i,j])))
    score_probs.sort(key=lambda x: x[1], reverse=True)
    out['top_scorelines'] = [{'score': s, 'prob': round(p,4)} for s,p in score_probs[:top_k]]

    if verbose:
        print(f"Predicción: {home_team} vs {away_team} (as_of={out['as_of']})")
        print(f"  λ_home={out['lambda_home']}  λ_away={out['lambda_away']}")
        print(f"  Prob 1X2 (logit): {out['prob_1X2']}")
        print(f"  Prob 1X2 (poisson grid): {out['prob_1X2_poisson']}")
        print("  Top marcadores:")
        for r in out['top_scorelines']:
            print(f"    {r['score']}: {r['prob']:.3%}")
    return out


In [101]:
# ======================= DASHBOARD: PREDICCIONES + ESTADÍSTICAS + TOP JUGADORES =======================
import warnings
warnings.filterwarnings("ignore", message="DataFrameGroupBy.apply operated on the grouping columns", category=FutureWarning)

import pandas as pd, numpy as np, math, html
from IPython.display import display, HTML

# ---------- CSS GLOBAL: quitar scroll en Classic Notebook, JupyterLab e ipywidgets ----------
_NO_SCROLL_CSS = """
<style>
/* Classic Notebook */
div.output_scroll { height:auto !important; max-height:none !important; overflow:visible !important; }
div.output_wrapper, div.output_area, div.output { overflow:visible !important; }
div.output_subarea { max-height:none !important; }

/* JupyterLab */
.jp-OutputArea, .jp-OutputArea-output { max-height:none !important; overflow:visible !important; }
.jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon * { overflow:visible !important; }

/* ipywidgets Output */
.jupyter-widgets-output-area, .jupyter-widgets-output-area * { max-height:none !important; overflow:visible !important; }

/* Nuestras tarjetas */
.dash-root, .dash-root * { overflow:visible !important; max-height:none !important; }
.team-card, .team-card * { overflow:visible !important; max-height:none !important; }
</style>
"""
display(HTML(_NO_SCROLL_CSS))

# ---------- helpers base ----------
def _to_datetime_safe(s): return pd.to_datetime(s, errors='coerce')
def _num(s): return pd.to_numeric(s, errors='coerce')

def _col(df, candidates):
    return next((c for c in candidates if c in df.columns), None)

def _first_col(df, candidates):
    for c in candidates:
        if c in df.columns: return c
    return None

# ---------- MF builders y H2H (robustos) ----------
def build_MF_from_sources(teams_long, matches_features=None):
    MF = matches_features
    if isinstance(MF, pd.DataFrame) and not MF.empty:
        df = MF.copy()
        h = _col(df, ['home_team','h_team','local','home','team_home'])
        a = _col(df, ['away_team','a_team','visitante','away','team_away'])
        d = _col(df, ['date','match_date','utc_date','kickoff'])
        if h and a and d:
            df = df.rename(columns={h:'home_team', a:'away_team', d:'date'})
            df['date'] = _to_datetime_safe(df['date'])
            for c in ['home_goals','h_goals','home_score','goals_home',
                      'away_goals','a_goals','away_score','goals_away']:
                if c in df.columns: df[c] = _num(df[c])
            ren = {}
            if 'h_goals' in df.columns and 'home_goals' not in df.columns: ren['h_goals']='home_goals'
            if 'home_score' in df.columns and 'home_goals' not in df.columns: ren['home_score']='home_goals'
            if 'goals_home' in df.columns and 'home_goals' not in df.columns: ren['goals_home']='home_goals'
            if 'a_goals' in df.columns and 'away_goals' not in df.columns: ren['a_goals']='away_goals'
            if 'away_score' in df.columns and 'away_goals' not in df.columns: ren['away_score']='away_goals'
            if 'goals_away' in df.columns and 'away_goals' not in df.columns: ren['goals_away']='away_goals'
            if ren: df.rename(columns=ren, inplace=True)
            keep = ['date','home_team','away_team'] + [c for c in ['home_goals','away_goals'] if c in df.columns]
            return df[keep].dropna(subset=['date','home_team','away_team'])
    TL = teams_long.copy() if isinstance(teams_long, pd.DataFrame) else pd.DataFrame()
    if TL.empty: return pd.DataFrame(columns=['date','home_team','away_team'])
    d = _col(TL, ['date','match_date','utc_date'])
    h = _col(TL, ['home_team','h_team','local','home','team_home'])
    a = _col(TL, ['away_team','a_team','visitante','away','team_away'])
    if d and h and a:
        df = TL[[d,h,a]].drop_duplicates().rename(columns={d:'date', h:'home_team', a:'away_team'})
        df['date'] = _to_datetime_safe(df['date'])
        return df[['date','home_team','away_team']].dropna()
    t = _col(TL, ['team','equipo','squad'])
    o = _col(TL, ['opponent','opp','rival','opposition'])
    s = _col(TL, ['side','h_a'])
    if d and t and o and s:
        tmp = TL.copy(); tmp['date'] = _to_datetime_safe(tmp[d])
        if s != 'side':
            ss = tmp[s].astype(str).str.lower()
            tmp['side'] = np.where(ss.eq('h'),'home', np.where(ss.eq('a'),'away', ss))
        homes = tmp[tmp['side'].astype(str).str.lower().eq('home')]
        if not homes.empty:
            return homes.rename(columns={t:'home_team', o:'away_team'})[['date','home_team','away_team']].dropna().drop_duplicates()
    return pd.DataFrame(columns=['date','home_team','away_team'])

def add_h2h_cols(MF):
    if MF is None or not isinstance(MF, pd.DataFrame) or MF.empty:
        return MF
    df = MF.copy()
    h = _col(df, ['home_team']); a = _col(df, ['away_team'])
    if not (h and a): return df
    df['__pair__'] = df[[h,a]].apply(lambda r: tuple(sorted([str(r[h]), str(r[a])])), axis=1)
    hg = _col(df, ['home_goals','h_goals','home_score','goals_home'])
    ag = _col(df, ['away_goals','a_goals','away_score','goals_away'])
    if hg: df[hg] = _num(df[hg])
    if ag: df[ag] = _num(df[ag])
    out = df.groupby('__pair__', dropna=False).size().rename('h2h_count').to_frame()
    if hg and ag:
        valid = df[hg].notna() & df[ag].notna()
        gdf = df.loc[valid].copy()
        if not gdf.empty:
            g = gdf.groupby('__pair__', dropna=False).agg(
                h2h_home_goals_avg = (hg, 'mean'),
                h2h_away_goals_avg = (ag, 'mean'),
                _home_wins = (hg, lambda s: np.mean(s.values > gdf.loc[s.index, ag].values) if len(s) else np.nan),
                _draws     = (hg, lambda s: np.mean(s.values == gdf.loc[s.index, ag].values) if len(s) else np.nan),
            )
            g['_away_wins'] = 1.0 - g['_home_wins'] - g['_draws']
            g = g.rename(columns={'_home_wins':'h2h_home_win_rate','_draws':'h2h_draw_rate','_away_wins':'h2h_away_win_rate'})
            out = out.join(g, how='left')
    df = df.merge(out, left_on='__pair__', right_index=True, how='left')
    return df.drop(columns=['__pair__'])

# ---------- compute_form_features (vectorizado) ----------
def compute_form_features(TL, window=5, half_life_days=60):
    if TL is None or not isinstance(TL, pd.DataFrame) or TL.empty:
        return pd.DataFrame(columns=['team','date','side','rest_days'])
    df = TL.copy()
    t = _col(df, ['team','equipo','squad'])
    d = _col(df, ['date','match_date','utc_date'])
    s = _col(df, ['side','h_a'])
    if not (t and d):
        return pd.DataFrame(columns=['team','date','side','rest_days'])
    if t != 'team': df = df.rename(columns={t:'team'})
    df['date'] = _to_datetime_safe(df[d])
    if s is None:
        df['side'] = 'any'
    else:
        if s != 'side':
            tmp = df[s].astype(str).str.lower()
            df['side'] = np.where(tmp.eq('h'),'home', np.where(tmp.eq('a'),'away', tmp))
    df = df[['team','date','side'] + [c for c in df.columns if c not in ['team','date','side']]]
    df = df.sort_values(['team','side','date'])
    df['rest_days'] = df.groupby(['team','side'], dropna=False)['date'].diff().dt.days
    cand_stats = [
        ('goals',      ['goals','gf']),
        ('opp_goals',  ['ga','conceded','against','opp_goals']),
        ('shots',      ['shots']),
        ('shots_ot',   ['shots_ot','sot','shots_on_target']),
        ('corners',    ['corners','corner_shots','corner']),
        ('pens',       ['pens','penalties']),
        ('points',     ['points','pts']),
        ('goals_home', ['goals_home','home_goals','h_goals','home_score']),
        ('shots_home', ['shots_home']),
        ('shots_ot_home',['shots_ot_home']),
        ('corners_home',['corners_home']),
        ('pens_home',  ['pens_home']),
        ('points_home',['points_home']),
        ('goals_away', ['goals_away','away_goals','a_goals','away_score']),
        ('shots_away', ['shots_away']),
        ('shots_ot_away',['shots_ot_away']),
        ('corners_away',['corners_away']),
        ('pens_away',  ['pens_away']),
        ('points_away',['points_away']),
    ]
    for base, syns in cand_stats:
        col = _col(df, syns)
        if not col: 
            continue
        x = _num(df[col])
        roll = x.groupby([df['team'], df['side']]).transform(lambda s: s.rolling(window, min_periods=1).mean())
        ewm  = x.groupby([df['team'], df['side']]).transform(lambda s: s.ewm(alpha=2.0/max(window,2), adjust=False).mean())
        df[f'roll_{base}_mean_{window}'] = roll
        df[f'ewm_{base}'] = ewm
        df[f'roll_{base}_w{window}'] = roll
        df[f'ewm_{base}_hl{half_life_days}'] = ewm
    return df

# ---------- utilidades para fallback de goles ----------
def _league_goals_means(MF):
    if MF is None or MF.empty: return 1.3, 1.3
    hg = _col(MF, ['home_goals']); ag = _col(MF, ['away_goals'])
    if not (hg and ag): return 1.3, 1.3
    return float(_num(MF[hg]).mean()), float(_num(MF[ag]).mean())

def _goals_series_from_TL(TL, team, side):
    if TL is None or TL.empty: return pd.Series([], dtype=float)
    df = TL.copy()
    if 'team' not in df.columns or 'date' not in df.columns or 'side' not in df.columns:
        return pd.Series([], dtype=float)
    df['date'] = _to_datetime_safe(df['date'])
    df = df[(df['team']==team) & (df['side']==side)].sort_values('date')
    cols = ['goals','gf']
    if side=='home':
        cols = ['goals','gf','goals_home','home_goals','h_goals','home_score']
    else:
        cols = ['goals','gf','goals_away','away_goals','a_goals','away_score']
    col = _col(df, cols)
    if not col: return pd.Series([], dtype=float)
    return _num(df[col]).dropna()

def _goals_series_from_MF(MF, team, side):
    if MF is None or MF.empty: return pd.Series([], dtype=float)
    df = MF.copy()
    h = _col(df, ['home_team']); a = _col(df, ['away_team'])
    hg = _col(df, ['home_goals']); ag = _col(df, ['away_goals'])
    if not (h and a and hg and ag): return pd.Series([], dtype=float)
    s = df[df[h]==team][hg] if side=='home' else df[df[a]==team][ag]
    return _num(s).dropna()

def _estimate_lambda(home_team, away_team, as_of, window=6):
    TL = teams_long.copy()
    TL['date'] = _to_datetime_safe(TL['date'])
    TL = TL[TL['date'] < _to_datetime_safe(as_of)]
    MF = build_MF_from_sources(teams_long, globals().get('matches_features', None))
    MF = MF[MF['date'] < _to_datetime_safe(as_of)].copy() if isinstance(MF, pd.DataFrame) and not MF.empty else MF
    s_h = _goals_series_from_TL(TL, home_team, 'home')
    if s_h.empty: s_h = _goals_series_from_MF(MF, home_team, 'home')
    s_a = _goals_series_from_TL(TL, away_team, 'away')
    if s_a.empty: s_a = _goals_series_from_MF(MF, away_team, 'away')
    lh0, la0 = _league_goals_means(MF)
    lh = float(s_h.tail(window).mean()) if not s_h.empty else lh0
    la = float(s_a.tail(window).mean()) if not s_a.empty else la0
    if not np.isfinite(lh) or lh<=0: lh = lh0
    if not np.isfinite(la) or la<=0: la = la0
    return lh, la

def _poisson_1x2_and_scores(lh, la, k=8, maxg=8):
    g = np.arange(0, maxg+1)
    P_h = np.exp(-lh) * np.power(lh, g) / np.vectorize(math.factorial)(g)
    P_a = np.exp(-la) * np.power(la, g) / np.vectorize(math.factorial)(g)
    M = np.outer(P_h, P_a)
    p_home = float(np.tril(M, -1).sum())
    p_draw = float(np.trace(M))
    p_away = float(np.triu(M, 1).sum())
    scores = [{'score': f'{i}-{j}', 'prob': float(M[i,j])}
              for i in range(maxg+1) for j in range(maxg+1)]
    scores = sorted(scores, key=lambda x: x['prob'], reverse=True)[:k]
    return {'home': p_home, 'draw': p_draw, 'away': p_away}, scores

# ---------- PREDICT MATCH (wrapper robusto) ----------
def predict_match_safe(home_team, away_team, as_of=None, top_k=8, verbose=False):
    if as_of is None:
        if 'teams_long' in globals() and isinstance(teams_long, pd.DataFrame) and not teams_long.empty and 'date' in teams_long.columns:
            as_of = _to_datetime_safe(teams_long['date']).max()
        else:
            as_of = pd.Timestamp.utcnow().normalize()
    as_of = _to_datetime_safe(as_of)
    if 'predict_match' in globals():
        try:
            out = predict_match(home_team, away_team, as_of=as_of, top_k=top_k, verbose=verbose)
            out.setdefault('home_team', home_team)
            out.setdefault('away_team', away_team)
            out.setdefault('as_of', str(as_of.date()))
            if 'lambda_home' in out and 'lambda_away' in out:
                p_poisson, tops = _poisson_1x2_and_scores(float(out['lambda_home']), float(out['lambda_away']), k=top_k)
                out.setdefault('prob_1X2_poisson', p_poisson)
                out.setdefault('top_scorelines', tops)
            if 'prob_1X2' not in out and 'prob_1X2_poisson' in out:
                out['prob_1X2'] = out['prob_1X2_poisson']
            return out
        except Exception:
            pass
    lh, la = _estimate_lambda(home_team, away_team, as_of, window=6)
    p_poisson, tops = _poisson_1x2_and_scores(lh, la, k=top_k)
    return {
        'home_team': home_team,
        'away_team': away_team,
        'as_of': str(as_of.date()),
        'lambda_home': round(float(lh), 3),
        'lambda_away': round(float(la), 3),
        'prob_1X2_poisson': p_poisson,
        'prob_1X2': p_poisson,
        'top_scorelines': tops
    }

# ---------- MÉTRICAS EXTRA (REEMPLAZO ROBUSTO CON DETECCIÓN "FUZZY") ----------
def _find_col_exact_or_fuzzy(df, exact_list, fuzzy_keywords=None):
    if df is None or df.empty: return None
    for c in exact_list:
        if c and c in df.columns:
            return c
    if fuzzy_keywords:
        lowmap = {c: c.lower() for c in df.columns}
        for c, cl in lowmap.items():
            for kw in fuzzy_keywords:
                if kw in cl:
                    return c
    return None

def _norm_team_matches(TL=None, MF=None):
    """
    Devuelve partidos en formato estándar:
    ['date','team','opponent','gf','ga','fouls','opp_fouls','yellows','reds']
    Soporta ancho (home/away) y largo (team/opponent). Detecta fouls/yellows/reds por nombres fuzzy.
    """
    TL = teams_long if TL is None else TL
    MF = globals().get('matches_features', None) if MF is None else MF

    def _from_wide(df):
        if df is None or df.empty: return pd.DataFrame()
        d  = _find_col_exact_or_fuzzy(df, ['date','match_date','utc_date','kickoff'])
        h  = _find_col_exact_or_fuzzy(df, ['home_team','h_team','local','home','team_home'])
        a  = _find_col_exact_or_fuzzy(df, ['away_team','a_team','visitante','away','team_away'])
        if not (d and h and a): return pd.DataFrame()
        hg = _find_col_exact_or_fuzzy(df, ['home_goals','h_goals','home_score','goals_home'])
        ag = _find_col_exact_or_fuzzy(df, ['away_goals','a_goals','away_score','goals_away'])
        hf = _find_col_exact_or_fuzzy(df, ['home_fouls','fouls_home','h_fouls'], fuzzy_keywords=['home_foul','local_foul','h_foul'])
        af = _find_col_exact_or_fuzzy(df, ['away_fouls','fouls_away','a_fouls'], fuzzy_keywords=['away_foul','visit_foul','a_foul'])
        hy = _find_col_exact_or_fuzzy(df, ['home_yellows','yellows_home','home_yellow_cards','home_crdy','home_yc','crd_y_home'],
                                      fuzzy_keywords=['home_yellow','home_crdy','home_crd_y','local_amarill'])
        ay = _find_col_exact_or_fuzzy(df, ['away_yellows','yellows_away','away_yellow_cards','away_crdy','away_yc','crd_y_away'],
                                      fuzzy_keywords=['away_yellow','away_crdy','away_crd_y','visit_amarill'])
        hr = _find_col_exact_or_fuzzy(df, ['home_reds','reds_home','home_red_cards','home_crdr','home_rc','crd_r_home'],
                                      fuzzy_keywords=['home_red','home_crdr','home_crd_r','local_roja'])
        ar = _find_col_exact_or_fuzzy(df, ['away_reds','reds_away','away_red_cards','away_crdr','away_rc','crd_r_away'],
                                      fuzzy_keywords=['away_red','away_crdr','away_crd_r','visit_roja'])
        use_cols = [c for c in [d,h,a,hg,ag,hf,af,hy,ay,hr,ar] if c]
        W = df[use_cols].copy().rename(columns={d:'date', h:'home_team', a:'away_team'})
        W['date'] = _to_datetime_safe(W['date'])
        for c in [hg,ag,hf,af,hy,ay,hr,ar]:
            if c in W.columns: W[c] = _num(W[c])
        home_rows = pd.DataFrame({
            'date':      W['date'],
            'team':      W['home_team'],
            'opponent':  W['away_team'],
            'gf':        W[hg] if hg in W.columns else np.nan,
            'ga':        W[ag] if ag in W.columns else np.nan,
            'fouls':     W[hf] if hf in W.columns else np.nan,
            'opp_fouls': W[af] if af in W.columns else np.nan,
            'yellows':   W[hy] if hy in W.columns else np.nan,
            'reds':      W[hr] if hr in W.columns else np.nan
        })
        away_rows = pd.DataFrame({
            'date':      W['date'],
            'team':      W['away_team'],
            'opponent':  W['home_team'],
            'gf':        W[ag] if ag in W.columns else np.nan,
            'ga':        W[hg] if hg in W.columns else np.nan,
            'fouls':     W[af] if af in W.columns else np.nan,
            'opp_fouls': W[hf] if hf in W.columns else np.nan,
            'yellows':   W[ay] if ay in W.columns else np.nan,
            'reds':      W[ar] if ar in W.columns else np.nan
        })
        out = pd.concat([home_rows, away_rows], ignore_index=True)
        out = out.dropna(subset=['date','team','opponent']).drop_duplicates(['date','team','opponent']).sort_values('date').reset_index(drop=True)
        for c in ['gf','ga','fouls','opp_fouls','yellows','reds']:
            out[c] = _num(out[c])
        return out

    wide_TL = _from_wide(TL)
    wide_MF = _from_wide(MF)
    if not wide_TL.empty or not wide_MF.empty:
        return pd.concat([wide_TL, wide_MF], ignore_index=True)

    # ---- Formato largo
    if TL is None or TL.empty:
        return pd.DataFrame(columns=['date','team','opponent','gf','ga','fouls','opp_fouls','yellows','reds'])

    d  = _find_col_exact_or_fuzzy(TL, ['date','match_date','utc_date','kickoff'])
    t  = _find_col_exact_or_fuzzy(TL, ['team','equipo','squad'])
    o  = _find_col_exact_or_fuzzy(TL, ['opponent','opp','rival','opposition'])
    if not (d and t and o):
        return pd.DataFrame(columns=['date','team','opponent','gf','ga','fouls','opp_fouls','yellows','reds'])

    gf = _find_col_exact_or_fuzzy(TL, ['goals','gf','goals_for','scored'])
    ga = _find_col_exact_or_fuzzy(TL, ['ga','conceded','against','opp_goals'])
    fo = _find_col_exact_or_fuzzy(TL, ['fouls','fouls_committed','fouls_for'], fuzzy_keywords=['foul','falta'])
    yl = _find_col_exact_or_fuzzy(TL, ['yellows','yellow_cards','yellow','crdy','yc'], fuzzy_keywords=['amarill','crd_y'])
    rd = _find_col_exact_or_fuzzy(TL, ['reds','red_cards','red','crdr','rc'],     fuzzy_keywords=['roja','crd_r'])

    base = TL[[d,t,o] + [c for c in [gf,ga,fo,yl,rd] if c]].copy().rename(columns={
        d:'date', t:'team', o:'opponent',
        (gf or 'gf'):'gf', (ga or 'ga'):'ga', (fo or 'fouls'):'fouls',
        (yl or 'yellows'):'yellows', (rd or 'reds'):'reds'
    })
    base['date'] = _to_datetime_safe(base['date'])
    for c in ['gf','ga','fouls','yellows','reds']:
        base[c] = _num(base.get(c))

    mirror = base.rename(columns={'team':'opponent','opponent':'team','fouls':'opp_fouls','gf':'ga_mirror'})
    out = base.merge(mirror[['date','team','opponent','opp_fouls','ga_mirror']],
                     on=['date','team','opponent'], how='left')
    out['ga'] = out['ga'].fillna(out['ga_mirror'])
    out = out.drop(columns=['ga_mirror'])
    return out[['date','team','opponent','gf','ga','fouls','opp_fouls','yellows','reds']].sort_values('date').reset_index(drop=True)

def _fmt_num(x, nd=2):
    if x is None or (isinstance(x,float) and not np.isfinite(x)): return "—"
    return f"{x:.{nd}f}"

def _team_extras_list(team, last_n=5, as_of=None):
    """
    Promedios de: faltas hechas/recibidas, amarillas, rojas, GF, GA y W–D–L
    en los últimos `last_n` (fallback a toda la temporada si esos N no tienen datos).
    """
    df = _norm_team_matches()
    if df.empty: return "<i>No hay datos.</i>"
    df['date'] = _to_datetime_safe(df['date'])
    if as_of is not None:
        df = df[df['date'] < _to_datetime_safe(as_of)]

    tdf = df[df['team']==team].sort_values('date')
    if tdf.empty: return "<i>No hay datos.</i>"

    last = tdf.tail(last_n).copy()

    def _mean_or_fallback(col):
        if col not in last.columns: return None
        s_last = _num(last[col]).dropna()
        if s_last.size: return float(s_last.mean())
        s_all = _num(tdf[col]).dropna()
        return float(s_all.mean()) if s_all.size else None

    fouls_for   = _mean_or_fallback('fouls')
    fouls_again = _mean_or_fallback('opp_fouls')
    yellows     = _mean_or_fallback('yellows')
    reds        = _mean_or_fallback('reds')
    gf_mean     = _mean_or_fallback('gf')
    ga_mean     = _mean_or_fallback('ga')

    # W–D–L
    gf_last = _num(last.get('gf')); ga_last = _num(last.get('ga'))
    mask = gf_last.notna() & ga_last.notna()
    if mask.sum() == 0:
        gf_use = _num(tdf.get('gf')); ga_use = _num(tdf.get('ga'))
        mask = gf_use.notna() & ga_use.notna()
        gf_use, ga_use = gf_use[mask], ga_use[mask]
    else:
        gf_use, ga_use = gf_last[mask], ga_last[mask]
    W = int((gf_use > ga_use).sum())
    D = int((gf_use == ga_use).sum())
    L = int((gf_use < ga_use).sum())
    n = int(mask.sum())

    return f"""
    <ul style="margin:0 0 0 18px;padding:0;font-size:13.5px">
      <li><b>Faltas hechas / partido:</b> {_fmt_num(fouls_for)}</li>
      <li><b>Faltas recibidas / partido:</b> {_fmt_num(fouls_again)}</li>
      <li><b>Amarillas / partido:</b> {_fmt_num(yellows)}</li>
      <li><b>Rojas / partido:</b> {_fmt_num(reds)}</li>
      <li><b>Goles a favor / partido:</b> {_fmt_num(gf_mean)}</li>
      <li><b>Goles en contra / partido:</b> {_fmt_num(ga_mean)}</li>
      <li><b>W–D–L:</b> {W}–{D}–{L} <span style="color:#666">(en {n} partidos)</span></li>
    </ul>
    """

# ---------- Tarjeta de equipo unificada (bloque original + métricas extra) ----------
def _card(title, inner_html):
    return f"""
    <div style="flex:1; min-width:320px; background:#fff; border:1px solid #e8e8e8;
                border-radius:12px; padding:14px; box-shadow:0 1px 4px rgba(0,0,0,0.04);">
      <div style="font-weight:600; margin-bottom:8px">{html.escape(title)}</div>
      <div>{inner_html}</div>
    </div>
    """

def _table(rows, headers):
    th = "".join(f"<th style='text-align:left;padding:6px 8px;border-bottom:1px solid #eee'>{html.escape(h)}</th>" for h in headers)
    trs = []
    for r in rows:
        tds = "".join(f"<td style='padding:6px 8px;border-bottom:1px solid #f6f6f6'>{c}</td>" for c in r)
        trs.append(f"<tr>{tds}</tr>")
    return f"<table style='border-collapse:collapse; font-size:13.5px; width:100%'>{'<thead><tr>'+th+'</tr></thead>'}<tbody>{''.join(trs)}</tbody></table>"

def _render_team_card_with_extras(team, bg="#f6f9ff", last_n=5, as_of=None):
    orig_html = ""
    if 'render_team_block' in globals():
        try:
            obj = render_team_block(team, bg)
            if isinstance(obj, HTML): orig_html = obj.data
            elif hasattr(obj, "_repr_html_"): orig_html = obj._repr_html_()
            else: orig_html = str(obj)
        except Exception:
            orig_html = ""
    extras = _team_extras_list(team, last_n=last_n, as_of=as_of)
    return HTML(f"""
    <div class="team-card" style="flex:1; min-width:380px; background:{bg}; border:1px solid #e8e8e8;
                border-radius:12px; padding:14px; box-shadow:0 1px 4px rgba(0,0,0,0.04);">
      <div style="display:flex; gap:16px; flex-wrap:wrap; align-items:flex-start">
        <div style="flex:2; min-width:320px">{orig_html or '<div style="color:#666"><i>Sin bloque de equipo preexistente.</i></div>'}</div>
        <div style="flex:1; min-width:260px">
          <div style="font-weight:600; margin-bottom:6px">Métricas extra</div>
          {extras}
        </div>
      </div>
    </div>
    """)

# ---------- BLOQUES DE RENDER (predicción) ----------
def _pct(x, d=1):
    try:
        if x is None or not np.isfinite(x): return "—"
        return f"{100*x:.{d}f}%"
    except Exception:
        return "—"

def render_prediction_block(home_team, away_team, as_of=None):
    out = predict_match_safe(home_team, away_team, as_of=as_of, top_k=8, verbose=False)
    p1 = f"""
    <div style="display:flex; gap:24px; flex-wrap:wrap">
      <div style="min-width:180px">
        <div style="font-size:12px;color:#666">λ goles (esperados)</div>
        <div style="margin-top:6px">
          <div><b>{html.escape(out['home_team'])}:</b> {out.get('lambda_home','—')}</div>
          <div><b>{html.escape(out['away_team'])}:</b> {out.get('lambda_away','—')}</div>
        </div>
      </div>
      <div style="min-width:220px">
        <div style="font-size:12px;color:#666">Prob. 1X2 (Logit)</div>
        <div style="margin-top:6px">
          <div>1 (local): <b>{_pct(out.get('prob_1X2',{}).get('home'))}</b></div>
          <div>X (empate): <b>{_pct(out.get('prob_1X2',{}).get('draw'))}</b></div>
          <div>2 (visitante): <b>{_pct(out.get('prob_1X2',{}).get('away'))}</b></div>
        </div>
      </div>
      <div style="min-width:220px">
        <div style="font-size:12px;color:#666">Prob. 1X2 (Poisson)</div>
        <div style="margin-top:6px">
          <div>1 (local): <b>{_pct(out.get('prob_1X2_poisson',{}).get('home'))}</b></div>
          <div>X (empate): <b>{_pct(out.get('prob_1X2_poisson',{}).get('draw'))}</b></div>
          <div>2 (visitante): <b>{_pct(out.get('prob_1X2_poisson',{}).get('away'))}</b></div>
        </div>
      </div>
    </div>
    <div style="font-size:12px;color:#666;margin-top:6px">as_of: {html.escape(str(out.get('as_of','—')))}</div>
    """
    card1 = _card("Predicción del partido", p1)
    tops = out.get('top_scorelines', []) or []
    rows = [[f"<b>{html.escape(r['score'])}</b>", _pct(r['prob'],2)] for r in tops]
    card2 = _card("Marcadores más probables", _table(rows, ["Marcador", "Probabilidad"]))

    try:
        preds_stats = _predict_team_stats(out['home_team'], out['away_team'], _to_datetime_safe(out.get('as_of')))
    except Exception:
        preds_stats = {}
    def _fmt(x): 
        return "—" if (x is None or (isinstance(x,float) and not np.isfinite(x))) else f"{x:.2f}"
    rows_stats = [
        ["Tiros",    _fmt(preds_stats.get('home_shots')),    _fmt(preds_stats.get('away_shots')),    _fmt(preds_stats.get('total_shots'))],
        ["Tiros OT", _fmt(preds_stats.get('home_shots_ot')), _fmt(preds_stats.get('away_shots_ot')), _fmt(preds_stats.get('total_shots_ot'))],
        ["Corners",  _fmt(preds_stats.get('home_corners')),  _fmt(preds_stats.get('away_corners')),  _fmt(preds_stats.get('total_corners'))],
        ["Faltas",   _fmt(preds_stats.get('home_fouls')),    _fmt(preds_stats.get('away_fouls')),    _fmt(preds_stats.get('total_fouls'))],
    ]
    card3 = _card("Estadísticas esperadas (equipo)", _table(rows_stats, ["Métrica", out['home_team'], out['away_team'], "Total"]))

    lam_h = float(out.get('lambda_home', 0.0) or 0.0)
    lam_a = float(out.get('lambda_away', 0.0) or 0.0)
    top_shots  = (globals()['_shots_top_players'](out['home_team'], out['away_team'], preds_stats, k=5) 
                  if '_shots_top_players' in globals() else None)
    top_fouls  = (globals()['_fouls_top_players'](out['home_team'], out['away_team'], preds_stats, k=5) 
                  if '_fouls_top_players' in globals() else None)
    top_goals  = (globals()['_goal_prob_top_players'](out['home_team'], out['away_team'], lam_h, lam_a, k=5) 
                  if '_goal_prob_top_players' in globals() else None)
    top_assist = (globals()['_assist_top_players'](out['home_team'], out['away_team'], lam_h, lam_a, k=5) 
                  if '_assist_top_players' in globals() else None)

    card4 = _card("Top 5 tiradores (esperado)", _table(top_shots or [["<i>No disponible</i>","",""]], ["Jugador","Equipo","Tiros esp."]))
    card5 = _card("Top 5 faltas (esperado)",   _table(top_fouls or [["<i>No disponible</i>","",""]], ["Jugador","Equipo","Faltas esp."]))
    card6 = _card("Top 5 goleadores probables",_table(top_goals or [["<i>No disponible</i>","","",""]], ["Jugador","Equipo","P(gol ≥ 1)","Goles esp."]))
    card7 = _card("Top 5 asistentes (esperado)",_table(top_assist or [["<i>No disponible</i>","",""]], ["Jugador","Equipo","Asist. esp."]))

    return HTML(f"""
      <div style="display:flex; gap:16px; flex-wrap:wrap">
        {card1}
        {card2}
      </div>
      <div style="height:8px"></div>
      <div style="display:flex; gap:16px; flex-wrap:wrap">
        {card3}
      </div>
      <div style="height:8px"></div>
      <div style="display:flex; gap:16px; flex-wrap:wrap">
        {card4}
        {card5}
        {card6}
        {card7}
      </div>
    """)

# ---------- agregadores recientes para stats de equipo usados en la tarjeta de predicción ----------
def _recent_side_avg_from_MF(team, side, as_of, stat, window=6):
    MF = globals().get('matches_features', None)
    if MF is None or not isinstance(MF, pd.DataFrame) or MF.empty: 
        return None
    MF = MF.copy()
    date_col = _first_col(MF, ['date', 'match_date'])
    if date_col is None: 
        return None
    MF[date_col] = _to_datetime_safe(MF[date_col])
    MF = MF[MF[date_col] < _to_datetime_safe(as_of)]
    home_team_col = _first_col(MF, ['home_team','h_team','local','home','team_home'])
    away_team_col = _first_col(MF, ['away_team','a_team','visitante','away','team_away'])
    if home_team_col is None or away_team_col is None:
        return None
    sk = 'home' if side=='home' else 'away'
    stat_syns = {
        'shots':    (f'{sk}_shots',),
        'shots_ot': (f'{sk}_shots_ot', f'{sk}_sot', f'{sk}_shots_on_target'),
        'corners':  (f'{sk}_corners', f'{sk}_corner_shots', f'{sk}_corner')
    }
    stat_col = _first_col(MF, stat_syns.get(stat, ()))
    if stat_col is None:
        return None
    mask = (MF[home_team_col]==team) if side=='home' else (MF[away_team_col]==team)
    s = _num(MF.loc[mask, stat_col]).dropna()
    if s.empty: return None
    return float(s.tail(window).mean())

def _recent_side_avg_from_TL(team, side, as_of, stat, window=6):
    TL = teams_long.copy()
    if 'date' not in TL.columns or 'side' not in TL.columns or 'team' not in TL.columns: return None
    TL['date'] = _to_datetime_safe(TL['date'])
    TL = TL[(TL['date'] < _to_datetime_safe(as_of)) & (TL['team']==team) & (TL['side']==side)]
    stat_syns = {
        'shots':    ('shots',),
        'shots_ot': ('shots_ot','sot','shots_on_target'),
        'corners':  ('corners','corner_shots','corner'),
        'fouls':    ('fouls',)
    }
    col = _first_col(TL, stat_syns.get(stat, ()))
    if col is None: return None
    s = _num(TL[col]).dropna()
    if s.empty: return None
    return float(s.tail(window).mean())

def _fouls_avg_from_players(team, as_of):
    pf = globals().get('df_player_fouls', None)
    if pf is None or not isinstance(pf, pd.DataFrame) or pf.empty: return None
    if 'team' not in teams_long.columns or 'date' not in teams_long.columns: return None
    TL = teams_long.copy()
    TL['date'] = _to_datetime_safe(TL['date'])
    team_games = TL[(TL['team']==team) & (TL['date'] < _to_datetime_safe(as_of))].shape[0]
    if team_games == 0: return None
    pf_team = pf[pf.get('team')==team]
    if pf_team.empty or 'fouls' not in pf_team.columns: return None
    fouls_tot = _num(pf_team['fouls']).sum()
    return float(fouls_tot / team_games)

def _predict_team_stats(home_team, away_team, as_of):
    preds = {}
    for side, team in [('home',home_team), ('away',away_team)]:
        for stat in ['shots','shots_ot','corners']:
            v = _recent_side_avg_from_TL(team, side, as_of, stat, window=6)
            if v is None:
                v = _recent_side_avg_from_MF(team, side, as_of, stat, window=6)
            if v is not None:
                preds[f"{side}_{stat}"] = float(v)
        v = _recent_side_avg_from_TL(team, side, as_of, 'fouls', window=6)
        if v is None:
            v = _fouls_avg_from_players(team, as_of)
        if v is not None:
            preds[f"{side}_fouls"] = float(v)
    if 'home_shots' in preds and 'away_shots' in preds:
        preds['total_shots'] = preds['home_shots'] + preds['away_shots']
    if 'home_shots_ot' in preds and 'away_shots_ot' in preds:
        preds['total_shots_ot'] = preds['home_shots_ot'] + preds['away_shots_ot']
    if 'home_corners' in preds and 'away_corners' in preds:
        preds['total_corners'] = preds['home_corners'] + preds['away_corners']
    if 'home_fouls' in preds and 'away_fouls' in preds:
        preds['total_fouls'] = preds['home_fouls'] + preds['away_fouls']
    return preds

# ---------- Cobertura del dataset ----------
def _dataset_coverage_card():
    TL = teams_long.copy()
    if TL.empty:
        return HTML(_card("Cobertura de datos", "<i>No hay datos.</i>"))
    d = _col(TL, ['date','match_date','utc_date'])
    TL['__date__'] = _to_datetime_safe(TL[d]) if d else pd.NaT
    round_col = _col(TL, ['round','matchday','jornada','gameweek','gw'])
    n_rounds = int(pd.Series(TL[round_col]).dropna().nunique()) if round_col else "—"
    h = _col(TL, ['home_team','h_team','local','home','team_home'])
    a = _col(TL, ['away_team','a_team','visitante','away','team_away'])
    if d and h and a:
        key = TL[[d,h,a]].dropna().copy()
        key.columns = ['date','home','away']
        key['date'] = _to_datetime_safe(key['date'])
        n_matches = int(key.drop_duplicates().shape[0])
    else:
        t = _col(TL, ['team','equipo','squad'])
        o = _col(TL, ['opponent','opp','rival','opposition'])
        if d and t and o:
            tmp = TL[[d,t,o]].dropna().copy()
            tmp['date'] = _to_datetime_safe(tmp[d])
            tmp['pair'] = tmp[[t,o]].apply(lambda r: tuple(sorted([str(r[t]), str(r[o])])), axis=1)
            n_matches = int(tmp.drop_duplicates(['date','pair']).shape[0])
        else:
            n_matches = "—"
    inner = f"""
    <div class="dash-root" style="display:flex;gap:28px;flex-wrap:wrap">
      <div><div style="font-size:12px;color:#666">Partidos totales</div>
           <div style="font-size:22px;font-weight:700;margin-top:2px">{n_matches}</div></div>
      <div><div style="font-size:12px;color:#666">Jornadas cubiertas</div>
           <div style="font-size:22px;font-weight:700;margin-top:2px">{n_rounds}</div></div>
    </div>
    """
    return HTML(_card("Cobertura de datos", inner))

# ---------- integración en el DASHBOARD (cobertura + tarjetas unificadas + predicción) ----------
def _render_dashboard_plus(home_team, away_team, as_of=None):
    blocks = []
    blocks.append(_dataset_coverage_card())
    blocks.append(HTML("<h3 style='margin:12px 0 6px 0'>Resumen equipos</h3>"))
    blocks += [
        HTML("<div class='dash-root' style='display:flex;gap:16px;flex-wrap:wrap'>"),
        _render_team_card_with_extras(home_team, "#f6f9ff", last_n=5, as_of=as_of),
        _render_team_card_with_extras(away_team,  "#fff6f6", last_n=5, as_of=as_of),
        HTML("</div>")
    ]
    blocks.append(HTML("<h3 style='margin:14px 0 6px 0'>Predicción</h3>"))
    blocks.append(render_prediction_block(home_team, away_team, as_of=as_of))
    display(*blocks)

# ---------- comprobaciones mínimas ----------
if 'teams_long' not in globals() or not isinstance(teams_long, pd.DataFrame) or teams_long.empty:
    raise SystemExit("Falta 'teams_long' en memoria. Ejecuta antes las celdas de carga.")

equipos = sorted(pd.Series(teams_long['team'].dropna(), dtype=str).unique().tolist())
if len(equipos) < 2:
    raise SystemExit("Hay menos de 2 equipos en 'teams_long'.")

# ---------- UI ----------
equipos = sorted(pd.Series(teams_long['team'].dropna(), dtype=str).unique().tolist())
try:
    import ipywidgets as W
    ddA = W.Dropdown(options=equipos, description='Local:', layout=W.Layout(width='350px'))
    ddB = W.Dropdown(options=equipos, description='Visitante:', layout=W.Layout(width='350px'))
    date_box = W.Text(value='', description='as_of:', placeholder='YYYY-MM-DD (opcional)', layout=W.Layout(width='250px'))
    btn = W.Button(description="Mostrar", button_style='primary')
    out = W.Output(layout=W.Layout(overflow='visible', max_height='none'))  # <- sin scroll

    def _on_click(_):
        with out:
            out.clear_output(wait=True)
            as_of = date_box.value.strip() or None
            _render_dashboard_plus(ddA.value, ddB.value, as_of=as_of)

    btn.on_click(_on_click)
    display(W.HBox([ddA, ddB, date_box, btn]), out)
    try: ddA.value, ddB.value = equipos[0], (equipos[1] if len(equipos)>1 else equipos[0])
    except Exception: pass
    _on_click(None)
except Exception:
    home, away = equipos[0], (equipos[1] if len(equipos)>1 else equipos[0])
    display(HTML("<div style='color:#a60'><b>Nota:</b> ipywidgets no está disponible, muestro un ejemplo estático.</div>"))
    _render_dashboard_plus(home, away, as_of=None)





Output(layout=Layout(max_height='none', overflow='visible'))